In [1]:
#~~~    Protocol
#~~~    EdgarFetch
#~~~ ~~~~    Routine
#~~~ ~~~~    Request-Filings-By-CIK

import datetime
start_time               = datetime.datetime.now()
job_log_suffix           = start_time.strftime("%y-%m-%d-%H-%M") #~~~ Used for uniqueness in multi-user environments (i.e. multiple reruns, by many people go to different directories, etc.)

import MySQLdb
import numpy
import os
import pprint
import requests
import sys
import urllib.request
from bs4 import BeautifulSoup

python_protocol          = 'EdgarFetch-V5'
job_log_prefix           = 'Request-Filings-By-CIK'

python_routine_long_name = (python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)

#
#~~~ ~~~~ 
#

edgarfetch_log_list                 = []
sec_edgar_doc_buttons_url_list      = []
sec_edgar_reply_metadata_list       = []

routine_config_sql_list             = []
numpy_routine_config_sql_array      = numpy.empty((0, 3))

creator_org_config_sql_list         = []
numpy_creator_org_config_sql_array  = numpy.empty((0, 3))
                           
hadoop_target_cluster_id            = 'CID-09b80c37-6a8f-4063-b2d0-255e56786283'

#
#~~~ ~~~~ 
#

routine_master_key             = 40000001               #~~~ routine_master_key:        EdgarFetch
routine_config_master_key      = 0                      #~~~ routine_config_master_key: Set for each run of EdgarFetch

record_content_type_key        = 2                      #~~~ 1 = External Data, 2 = External Data, 3 = IoT Data

#
#~~~ ~~~~ 
#
source_org_config_master_key    = 3300080                #~~~ source_org_config_master_key for:   U.S. SEC EDGAR System

creator_org_config_master_key   = 0                      #~~~ creator_org_config_master_key:  Initialized - Always zero here

#creator_org_config_master_key   = 0                      #~~~ creator_org_config_master_key used for unknown Org's


#creator_org_master_key          = 30000018               #~~~ creator_org_master_key for: Chameleon Gems Inc.
#sec_company_cik                 = '0001385807'           #~~~ CIK for Chameleon Gems Inc. (Make sure to include leading zeros)

#creator_org_master_key          = 30000019               #~~~ creator_org_master_key for: Lizard International Fund LLC
#sec_company_cik                 = '0001458864'           #~~~ CIK for Lizard International Fund LLC (Make sure to include leading zeros)

creator_org_master_key          = 30000014               #~~~ creator_org_master_key for: Wave Life Sciences
sec_company_cik                 = '0001631574'           #~~~ CIK for Wave Life Sciences (Make sure to include leading zeros)



#
#~~~ ~~~~ 
#

print('>>>>>>>>>>>>>>>>>>>>>>------<<<<<<<<<<<<<<<<<<<<<<<<<<<<')
print('>>>>>  P R O T O C O L   E X E C U T I O N   B E G I N S')
print('>>>>>>>>>>>>>>>>>>>>>>------<<<<<<<<<<<<<<<<<<<<<<<<<<<<')
print('>>>>>')
print('>>>>>      P R O T O C O L: ', python_protocol)
print('>>>>>      _____________________________________________')
print('>>>>>')
print('>>>>>        JOB ID Set To: ', job_log_suffix)
print('>>>>>      _____________________________________________')
print('>>>>>')
print('>>>>>           Start Time: ', str(start_time))
print('>>>>>')
print('>>>>>>>>>>>>>>>>>>>>>>------<<<<<<<<<<<<<<<<<<<<<<<<<<<<')
print(' ')

edgarfetch_log_list.append('>>>>>>>>>>>>>>>>>>>>>>------<<<<<<<<<<<<<<<<<<<<<<<<<<<<')
edgarfetch_log_list.append('>>>>>  P R O T O C O L   E X E C U T I O N   B E G I N S')
edgarfetch_log_list.append('>>>>>>>>>>>>>>>>>>>>>>------<<<<<<<<<<<<<<<<<<<<<<<<<<<<')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>      P R O T O C O L: ' + python_protocol)
edgarfetch_log_list.append('>>>>>      _____________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>        JOB ID Set To: ' + job_log_suffix)
edgarfetch_log_list.append('>>>>>      _____________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>           Start Time: ' + str(start_time))
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>>>>>>>>>>>>>>>>>>------<<<<<<<<<<<<<<<<<<<<<<<<<<<<')
edgarfetch_log_list.append(' ')

                           
print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>            Start Time: ' + str(datetime.datetime.now()))
print('_____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>        Routine Starts: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append(' ')

#
#~~~ ~~~~ 1.1.2 Set MySQL Connection Parameters
#

mysql_01_host       = 'www.chameleonmetadata.com'
mysql_01_port       = 3306
mysql_01_user       = 'ericth5_adm2018h'
mysql_01_pw         = '@password4adm2018h'
mysql_01_db         = 'ericth5_chameleon_2018h'
                         
                           
print('>>>>>         mysql_01_host       = www.chameleonmetadata.com')
print('>>>>>         mysql_01_port       = 3306')
print('>>>>>         mysql_01_user       = ericth5_adm2018h')
print('>>>>>         mysql_01_database   = ericth5_chameleon_2018h')
print('_____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>         mysql_01_host       = www.chameleonmetadata.com')
edgarfetch_log_list.append('>>>>>         mysql_01_port       = 3306') 
edgarfetch_log_list.append('>>>>>         mysql_01_user       = ericth5_adm2018h')
edgarfetch_log_list.append('>>>>>         mysql_01_database   = ericth5_chameleon_2018h')
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append(' ')

#
#~~~ ~~~~ 1.1.3 Set Storage Hierarchy Parameters
#

storage_hierarchy_name            = 'EdgarFetch'

local_server_os                   = 'Windows'              #~~~~ Operating System (not used when building directory names)
local_server_ip                   = '192.168.1.114'        #~~~~ IP of Server which will store the Edgar datasets
windows_server_name               = '192-168-1-114'        #~~~~ Windows Network Server Name
local_root_drive                  = 'x'                    #~~~~ Root drive on server which will store the Edgar datasets
dir_purpose                       = 'dmz-ftp'              #~~~~ 1st directory level (i.e. x:/dmz-ftp)
dir_source                        = 'sec-edgar'            #~~~~ 2nd directory level (i.e. x:/dmz-ftp/sec-edgar)
dir_source_subgroup_01            = sec_company_cik        #~~~~ 3rd directory level (i.e. x:/dmz-ftp/sec-edgar/cik)
dir_source_subgroup_02            = 'secNum'               #~~~~ 4th directory level (i.e. x:/dmz-ftp/sec-edgar/cik/secNum)


print('Storage Hierarchy (' + storage_hierarchy_name + ') with Nodes Set As:')
print(' ~ local_server_os         = ' + local_server_os)
print(' ~ ~ local_server_ip         = ' + local_server_ip)
print(' ~ ~ ~ windows_server_name     = ' + windows_server_name)
print(' ~ ~ ~ ~ local_root_drive        = ' + local_root_drive)
print(' ~ ~ ~ ~ ~ dir_purpose              = ' + dir_purpose)
print(' ~ ~ ~ ~ ~ ~ dir_source               = ' + dir_source)
print(' ~ ~ ~ ~ ~ ~ ~ dir_source_subgroup_01   = ' + dir_source_subgroup_01)
print(' ~ ~ ~ ~ ~ ~ ~ ~ dir_source_subgroup_02   = ' + dir_source_subgroup_02)
print(' ~ ~ ~ ~ ~ ~ ~ ~ ~ JOB ID                   = ' + job_log_suffix)
print('_____________________________________________________________________ \n')

edgarfetch_log_list.append('Storage Hierarchy (' + storage_hierarchy_name + ') with Nodes Set As:')
edgarfetch_log_list.append(' ~ local_server_os         = ' + local_server_os)
edgarfetch_log_list.append(' ~ ~ local_server_ip         = ' + local_server_ip)
edgarfetch_log_list.append(' ~ ~ ~ windows_server_name     = ' + windows_server_name)
edgarfetch_log_list.append(' ~ ~ ~ ~ local_root_drive        = ' + local_root_drive)
edgarfetch_log_list.append(' ~ ~ ~ ~ ~ dir_purpose              = ' + dir_purpose)
edgarfetch_log_list.append(' ~ ~ ~ ~ ~ ~ dir_source               = ' + dir_source)
edgarfetch_log_list.append(' ~ ~ ~ ~ ~ ~ ~ dir_source_subgroup_01   = ' + dir_source_subgroup_01)
edgarfetch_log_list.append(' ~ ~ ~ ~ ~ ~ ~ ~ dir_source_subgroup_02   = ' + dir_source_subgroup_02)
edgarfetch_log_list.append(' ~ ~ ~ ~ ~ ~ ~ ~ ~ JOB ID                   = ' + job_log_suffix)
edgarfetch_log_list.append('_____________________________________________________________________ ')
edgarfetch_log_list.append(' ')

#
#~~~ ~~~~
#

sec_edgar_request_url_prefix      = 'https://www.sec.gov'
sec_edgar_document_request_prefix = 'Archives/edgar/data'

filing_type                       = '10-K'                 #~~~~ &type  URL variable / Format: 'XXX' (Filter on type of filing)
#filing_type                       = ' '
until_date                        = ' '                    #~~~~ &dateb URL variable / Format: 'YYYYMMDD' (Filter on maximum filing date requested)
filing_count                      = ' '                   #~~~~ &count URL variable / Format: '99'  (Limit the number of filings to return)

sec_edgar_request_url             = ('https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=' 
                                     + sec_company_cik)

if not filing_type                == ' ':
    sec_edgar_request_url         = (sec_edgar_request_url
                                     + '&type=' 
                                     + filing_type)
    
if not until_date                 == ' ':
    sec_edgar_request_url         = (sec_edgar_request_url 
                                     + '&dateb=' 
                                     + until_date)

if not filing_count               == ' ':
    sec_edgar_request_url         = (sec_edgar_request_url 
                                     + '&count=' 
                                     + filing_count)   

                           
print('Request Sent to SEC Edgar: \n')
print(sec_edgar_request_url)
print('_____________________________________________________________________ \n') 
                           
edgarfetch_log_list.append('Request Sent to SEC Edgar:')
edgarfetch_log_list.append(' ')
edgarfetch_log_list.append(sec_edgar_request_url)    
edgarfetch_log_list.append('_____________________________________________________________________') 
edgarfetch_log_list.append(' ')

#
#~~~ ~~~~
#

                                       
wss_routine_config_master_name         = ( python_protocol + '_'
                                         + sec_company_cik + '_'
                                         + job_log_suffix) 
                                       
output_os_dir_purpose_src_prefix       = (local_root_drive + ':' + '/' 
                                          + dir_purpose + '/'
                                          + dir_source)
                                       
output_os_directory_cik_prefix         = (local_root_drive + ':' + '/'
                                          + dir_purpose + '/'
                                          + dir_source + '/'
                                          + sec_company_cik)
                                       
hadoop_purpose_source1_prefix          = (dir_purpose + '/'
                                          + dir_source)
                                       
hadoop_copyFromLocal_prefix            = ('bin/hadoop fs -copyFromLocal "//' 
                                          + local_server_ip + '/'
                                          + local_root_drive + '/' 
                                          + dir_purpose + '/'
                                          + dir_source) 
                                       
hadoop_mkdir_command_prefix            = ('bin/hadoop fs -mkdir /')
                                       
output_os_replyDocButtons_file         = (local_root_drive + ':' + '/' 
                                          + dir_purpose + '/'
                                          + dir_source + '/'
                                          + sec_company_cik + '/'
                                          + python_protocol + '_'
                                          + job_log_suffix + '_replyDocButtons.metadata')
                                       
output_os_hadoop_mkdir_cmds_file       = (local_root_drive + ':' + '/' 
                                          + dir_purpose + '/'
                                          + dir_source + '/'
                                          + sec_company_cik + '/'
                                          + python_protocol + '_'
                                          + job_log_suffix + '_mkdir.hadoop')
                                       
edgar_fetch_log_file                   = (local_root_drive + ':' + '/' 
                                          + dir_purpose + '/'
                                          + dir_source + '/'
                                          + sec_company_cik + '/' 
                                          + python_protocol + '_'
                                          + job_log_suffix + '.log')
                                       
doc_buttons_write_dir                  = (output_os_dir_purpose_src_prefix + '/' 
                                         + sec_company_cik) 
                                       
sec_edgar_request_url_file             = (output_os_dir_purpose_src_prefix + '/' 
                                         + sec_company_cik + '/' 
                                         + python_protocol + '_'
                                         + job_log_suffix + '.request')      
                                       
sec_edgar_reply_url_file               = (output_os_dir_purpose_src_prefix + '/' 
                                         + sec_company_cik + '/' 
                                         + python_protocol + '_'
                                         + job_log_suffix + '.reply')
                                       
routine_facet_value_sql_file           = (output_os_dir_purpose_src_prefix + '/' 
                                         + sec_company_cik + '/' 
                                         + python_protocol + '_'
                                         + job_log_suffix + '_facet-values-routine.sql')
                                       
dataset_facet_value_sql_file           = (output_os_dir_purpose_src_prefix + '/' 
                                         + sec_company_cik + '/' 
                                         + python_protocol + '_'
                                         + job_log_suffix + '_facet-values-dataset.sql')
                                       
creator_org_facet_value_sql_file       = (output_os_dir_purpose_src_prefix + '/' 
                                         + sec_company_cik + '/' 
                                         + python_protocol + '_'
                                         + job_log_suffix + '_facet-values-org.sql')
                                       
org_config_sql_file                    = (output_os_dir_purpose_src_prefix + '/' 
                                         + sec_company_cik + '/' 
                                         + python_protocol + '_'
                                         + job_log_suffix + '_config-orgs.sql')
                                       
routine_config_sql_file                = (output_os_dir_purpose_src_prefix + '/' 
                                         + sec_company_cik + '/' 
                                         + python_protocol + '_'
                                         + job_log_suffix + '_config_routines.sql')
                                       
dataset_config_sql_file                = (output_os_dir_purpose_src_prefix + '/' 
                                         + sec_company_cik + '/' 
                                         + python_protocol + '_'
                                         + job_log_suffix + '_config_dataset.sql')
#
#~~~ ~~~~
#

with urllib.request.urlopen(sec_edgar_request_url) as edgar_request_url:
    edgar_request_url_read = edgar_request_url.read()
                           
#
#~~~ ~~~~ 
#

    soup_edgar_request              = BeautifulSoup(edgar_request_url_read, "html.parser") 
    soup_edgar_companyNames         = soup_edgar_request.find_all('span', {"class": "companyName"})
    edgar_companyNames_count        = 0
    
    for edgar_companyName in soup_edgar_companyNames:
                           
        edgar_companyNames_count    = edgar_companyNames_count + 1
        
        if edgar_companyNames_count > 1:
            edgarfetch_log_list.append('ERROR: Request to Edgar Returned Multiple Campany Names for A Single CIK - Execution Aborting')
            sys.exit("ERROR: Request to Edgar Returned Multiple Campany Names for A Single CIK - Execution Aborting") 
                           
        edgar_companyName_text      = edgar_companyName.text
        edgar_companyName_text      = edgar_companyName_text.replace('\n', '') 
        edgar_companyName_text      = edgar_companyName_text.replace(' (see all company filings)', '') 
    
        edgarfetch_log_list.append('Company Name: ' + edgar_companyName_text)
        edgarfetch_log_list.append('_____________________________________________________________________')
        edgarfetch_log_list.append(' ')
                                               
#
#~~~ ~~~~ 
#        
        
    edgarfetch_log_list.append('Reply URL List Received from SEC Edgar (Documents Button Target URL): ')
    edgarfetch_log_list.append(' ')

    soup_edgar_request_tables = soup_edgar_request.find_all('table')
    
    for soup_table in soup_edgar_request_tables:
        
        href_list = soup_edgar_request.find_all('a', id='documentsbutton')

        for href in href_list:
            doc_button_href     = href['href']
            sec_edgar_url       = (sec_edgar_request_url_prefix + doc_button_href)
            
            if sec_edgar_url not in sec_edgar_doc_buttons_url_list:
                sec_edgar_doc_buttons_url_list.append(sec_edgar_url)

                edgarfetch_log_list.append(sec_edgar_url)

    edgarfetch_log_list.append('_____________________________________________________________________') 
    edgarfetch_log_list.append(' ')

#
#~~~ ~~~~ 
#

    edgarfetch_log_list.append('EdgarFetch - Creator Org Configuration INSERT Statement: ')
    edgarfetch_log_list.append(' ')
    
    org_config_select_string               = ('SELECT org_config_master_key FROM organization_config_master WHERE org_config_master_name = ' + "'" + edgar_companyName_text + "'")
    
    if creator_org_master_key == 0:
        org_config_master_notes            = 'DATA GOVERNANCE ALERT: No Valid (creator_org_master_key) was provided at runtime.  Setting (creator_org_master_key) = 0 for further investigation' 
        org_config_insert_string           = ('INSERT INTO organization_config_master VALUES (NULL , ' + str(3) + ' , 0, 0, NULL, CURRENT_TIMESTAMP, NULL' + ', ' + "'" + edgar_companyName_text + "'" + ', ' + "'" + org_config_master_notes + "'" + ')')
    else:
        org_config_insert_string           = ('INSERT INTO organization_config_master VALUES (NULL , ' + str(3) + ' , 0, 0, NULL, CURRENT_TIMESTAMP, NULL' + ', ' + "'" + edgar_companyName_text + "'" + ', NULL)')

    try: 
        org_config_creator_sql_array       = numpy.array([str(creator_org_master_key), org_config_select_string , org_config_insert_string])           
        numpy_creator_org_config_sql_array = numpy.append(numpy_creator_org_config_sql_array, [org_config_creator_sql_array], axis=0)

        edgarfetch_log_list.append(org_config_insert_string)
        edgarfetch_log_list.append('_____________________________________________________________________')
     
    except:
        edgarfetch_log_list.append('ERROR: EdgarFetch - Routine Configuration INSERT Statement - Execution Aborting')
        edgarfetch_log_list.append(' ~ DUMP: routine_master_key = ' + str(routine_master_key))
        edgarfetch_log_list.append(' ~ DUMP: ' + routine_config_select_string)
        edgarfetch_log_list.append(' ~ DUMP: ' + routine_config_insert_string) 
        sys.exit("ERROR: EdgarFetch - Routine Configuration INSERT Statement - Execution Aborting")
        
    edgarfetch_log_list.append(' ')
                      
#
#~~~ ~~~~ 2.2.5 Build & Capture (routine_config_master) INSERT Statement
#   
    edgarfetch_log_list.append('EdgarFetch - Routine Configuration INSERT Statement: ')
    edgarfetch_log_list.append(' ')
    
    routine_config_notes                   = python_protocol + '_' + job_log_suffix + ' U.S. SEC Filings for: ' + edgar_companyName_text

    routine_config_select_string           = ('SELECT routine_config_master_key FROM routine_config_master WHERE routine_config_master_name = ' + "'" + wss_routine_config_master_name + "'")        
    routine_config_insert_string           = ('INSERT INTO routine_config_master VALUES (NULL , ' + str(57) + ' , 0, 0, CURRENT_TIMESTAMP, NULL, NULL, ' + "'" + wss_routine_config_master_name + "'" + ', ' + "'" + routine_config_notes + "'" + ')')
    
    try:
        routine_config_sql_array           = numpy.array([str(routine_master_key), routine_config_select_string , routine_config_insert_string])
        numpy_routine_config_sql_array     = numpy.append(numpy_routine_config_sql_array, [routine_config_sql_array], axis=0)
    
        edgarfetch_log_list.append(routine_config_insert_string)
        edgarfetch_log_list.append('_____________________________________________________________________')
        edgarfetch_log_list.append(' ')

    except:
        edgarfetch_log_list.append('ERROR: EdgarFetch - Routine Configuration INSERT Statement - Execution Aborting')
        edgarfetch_log_list.append(' ~ DUMP: routine_master_key = ' + str(routine_master_key))
        edgarfetch_log_list.append(' ~ DUMP: ' + routine_config_select_string)
        edgarfetch_log_list.append(' ~ DUMP: ' + routine_config_insert_string) 
        sys.exit("ERROR: EdgarFetch - Routine Configuration INSERT Statement - Execution Aborting")

#
#~~~ ~~~~
#

if not os.path.exists(doc_buttons_write_dir):
    print('Target Directory Not Found:     ', doc_buttons_write_dir)
    edgarfetch_log_list.append('Target Directory Not Found:')
    edgarfetch_log_list.append(' ')
    edgarfetch_log_list.append(' ~ ' + doc_buttons_write_dir)
    try:
        os.makedirs(doc_buttons_write_dir)
        print(' ~ Target Directory Created:    ', doc_buttons_write_dir)
        edgarfetch_log_list.append(' ~ Target Directory Created   ')
        edgarfetch_log_list.append(' ~ ~ ' + doc_buttons_write_dir)
    except:
        print(' ~ ERROR: Creating Directory')
        edgarfetch_log_list.append(' ~ ~ ERROR: Creating Target Directory')
else:
    print('Target Directory Already Exists: ', doc_buttons_write_dir)
    edgarfetch_log_list.append('Target Directory Already Exists: ')
    edgarfetch_log_list.append(' ')
    edgarfetch_log_list.append(' ~ ' + doc_buttons_write_dir)

print('_____________________________________________________________________ \n') 
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append(' ')
        

#
#~~~ ~~~~ Python Routine Termination
#

print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>          Completed at: ' + str(datetime.datetime.now()))
print('_____________________________________________________________________')
print('_____________________________________________________________________')
print('_____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>          Completed at: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')


#
#~~~ ~~~~ MIT Creative Commons License
#
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------      Chameleon Metadata Python Software     -------------------------------------------
#------------------------------    © 2018 Chameleon Metadata (https://ChameleonMetadata.com)    -------------------------------- 
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------   MIT Creative Commons License   ------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------
# Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated  
# documentation files (the "Software"), to deal in the Software without restriction, including without limitation 
# the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and 
# to permit persons to whom the Software is furnished to do so, subject to the following conditions:
#
# 1) The above copyright notice & this permission notice shall be included in all copies or substantial portions of the Software.
#
# 2) The Software is provided "as is", without warranty of any kind, express or implied, including but not limited to 
#    the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors 
#    or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or 
#    otherwise, arising from, out of or in connection with the software or the use or other dealings in the Software.
#-------------------------------------------------------------------------------------------------------------------------------

>>>>>>>>>>>>>>>>>>>>>>------<<<<<<<<<<<<<<<<<<<<<<<<<<<<
>>>>>  P R O T O C O L   E X E C U T I O N   B E G I N S
>>>>>>>>>>>>>>>>>>>>>>------<<<<<<<<<<<<<<<<<<<<<<<<<<<<
>>>>>
>>>>>      P R O T O C O L:  EdgarFetch-V5
>>>>>      _____________________________________________
>>>>>
>>>>>        JOB ID Set To:  18-12-10-12-50
>>>>>      _____________________________________________
>>>>>
>>>>>           Start Time:  2018-12-10 12:50:31.037270
>>>>>
>>>>>>>>>>>>>>>>>>>>>>------<<<<<<<<<<<<<<<<<<<<<<<<<<<<
 
>>>>>
>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S
>>>>>  _______________________________________________
>>>>>
>>>>>         R O U T I N E: EdgarFetch-V5.Request-Filings-By-CIK.18-12-10-12-50
>>>>>            Start Time: 2018-12-10 12:50:31.537292
_____________________________________________________________________ 

>>>>>         mysql_01_host       = www.chameleonmetadata.com
>>>>>         mysql_01_port       = 3306
>>>>>         mysql_01_user       = ericth5_adm2018h
>>

In [2]:
#~~~    Protocol
#~~~    EdgarFetch
#~~~ ~~~~    Routine
#~~~ ~~~~    Load-EdgarFetch-Class-Facet-Copybook
#

job_log_prefix                 = 'Load-EdgarFetch-Class-Facet-Copybook'
python_routine_long_name       = (python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)

print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>            Start Time: ' + str(datetime.datetime.now()))
print('>>>>> _____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>        Routine Starts: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append(' ')

#
#~~~ ~~~~ Set Classification Facet Variables for EdgarFetch (Pre-assigned in Chameleon Metadata before execution)
#

class_facet_8000906               = 8000906   #~~~ #~~~ Facet Key for: Subject to Privacy Laws
cfv_8000906_FACTA                         = 7019857               #~~~ #~~~ #~~~
cfv_8000906_FACTA_alpha                   = 'United States: Fair and Accurate Credit Transactions Act of 2003 (FACTA)'
cfv_8000906_CPNI                          = 7019854               #~~~ #~~~ #~~~
cfv_8000906_CPNI_alpha                    = 'United States: Customer Proprietary Network Information (CPNI)'
#cfv_8000906_COPPA                         = 7019856               #~~~ #~~~ #~~~
#cfv_8000906_HIPAA                         = 7019855               #~~~ #~~~ #~~~
#cfv_8000906_Philippine_Republic_Act_10173 = 7019853               #~~~ #~~~ #~~~
#cfv_8000906_UK_Data_Protection_Act        = 7019852               #~~~ #~~~ #~~~
#cfv_8000906_UK_GDPR                       = 7019851               #~~~ #~~~ #~~~
#cfv_8000906_CH_STE_108_Only               = 7019850               #~~~ #~~~ #~~~
#cfv_8000906_CH_GDPR                       = 7019849               #~~~ #~~~ #~~~
#cfv_8000906_FR_Law78-17                   = 7019848               #~~~ #~~~ #~~~
#cfv_8000906_EU_GDPR                       = 7019847               #~~~ #~~~ #~~~
#cfv_8000906_Canada_PIPEDA                 = 7019846               #~~~ #~~~ #~~~

class_facet_8000850               = 8000850   #~~~ #~~~ Facet Key for: Dataset Content - Spoken Language of Content
cfv_8000850_english                 = 7016098               #~~~ #~~~ #~~~
cfv_8000850_english_alpha           = 'English'             #~~~ #~~~ #~~~

class_facet_8000830               = 8000830   #~~~ #~~~ Facet Key for: Dataset Storage Platform
cfv_8000830_WindowsServer2012       = 7016289               #~~~ #~~~ #~~~
cfv_8000830_WindowsServer2012_alpha = 'Windows Server 2012' #~~~ #~~~ #~~~


class_facet_8000888               = 8000888   #~~~ #~~~ Facet Key for: Software Protocol
class_facet_8000889               = 8000889   #~~~ #~~~ Facet Key for: Software Routine
class_facet_8000890               = 8000890   #~~~ #~~~ Facet Key for: Job Log Prefix

class_facet_8000857               = 8000857   #~~~ #~~~ Facet Key for: Hadoop/HDFS Cluster ID
class_facet_8000861               = 8000861   #~~~ #~~~ Facet Key for: Hadoop HDFS Directory
class_facet_8000862               = 8000862   #~~~ #~~~ Facet Key for: Windows Directory
class_facet_8000870               = 8000870   #~~~ #~~~ Facet Key for: Storage Hierarchy Name
class_facet_8000871               = 8000871   #~~~ #~~~ Facet Key for: local_server_os
class_facet_8000872               = 8000872   #~~~ #~~~ Facet Key for: local_server_ip
class_facet_8000887               = 8000887   #~~~ #~~~ Facet Key for: local_root_drive
class_facet_8000873               = 8000873   #~~~ #~~~ Facet Key for: dir_purpose
class_facet_8000874               = 8000874   #~~~ #~~~ Facet Key for: dir_source
class_facet_8000875               = 8000875   #~~~ #~~~ Facet Key for: dir_source_subgroup_01
class_facet_8000876               = 8000876   #~~~ #~~~ Facet Key for: dir_source_subgroup_02
class_facet_8000877               = 8000877   #~~~ #~~~ Facet Key for: Job Log Suffix (used as Storage Hierarchy JOB ID)
class_facet_8000886               = 8000886   #~~~ #~~~ Facet Key for: Operating System Name

class_facet_8000895               = 8000895   #~~~ #~~~ Facet Key for: MySQL host
class_facet_8000896               = 8000896   #~~~ #~~~ Facet Key for: MySQL port
class_facet_8000897               = 8000897   #~~~ #~~~ Facet Key for: MySQL userid
class_facet_8000898               = 8000898   #~~~ #~~~ Facet Key for: MySQL database name

class_facet_8000880               = 8000880   #~~~ #~~~ Facet Key for: EDGAR Filing: Filing Form
class_facet_8000881               = 8000881   #~~~ #~~~ Facet Key for: EDGAR Filing: Accession Number
class_facet_8000882               = 8000882   #~~~ #~~~ Facet Key for: EDGAR Filing: &type (i.e. filing type)
class_facet_8000885               = 8000885   #~~~ #~~~ Facet Key for: EDGAR Filing: Central Index Key (CIK)
class_facet_8000899               = 8000899   #~~~ #~~~ Facet Key for: EDGAR Filing: Filing Date
class_facet_8000900               = 8000900   #~~~ #~~~ Facet Key for: EDGAR Filing: Accepted Date & Time
class_facet_8000901               = 8000901   #~~~ #~~~ Facet Key for: EDGAR Filing: Documents
class_facet_8000902               = 8000902   #~~~ #~~~ Facet Key for: EDGAR Filing: Period of Report

class_facet_8000892               = 8000892   #~~~ #~~~ Facet Key for: targetDirectoryFtpDmz
class_facet_8000893               = 8000893   #~~~ #~~~ Facet Key for: requestSentURL
class_facet_8000894               = 8000894   #~~~ #~~~ Facet Key for: replyDocButtonURL
class_facet_8000903               = 8000903   #~~~ #~~~ Facet Key for: Dataset Collection Name
class_facet_8000905               = 8000905   #~~~ #~~~ Facet Key for: Dataset Description

class_facet_8000194               = 8000194   #~~~ #~~~ Facet Key for: Python Package Name
cfv_8000194_BeautifulSoup           = 7018755               #~~~ #~~~ #~~~
cfv_8000194_datetime                = 7018316               #~~~ #~~~ #~~~
cfv_8000194_MySQLdb                 = 7018440               #~~~ #~~~ #~~~
cfv_8000194_numpy                   = 7018460               #~~~ #~~~ #~~~
cfv_8000194_os                      = 7018253               #~~~ #~~~ #~~~
cfv_8000194_pprint                  = 7018501               #~~~ #~~~ #~~~
cfv_8000194_requests                = 7018757               #~~~ #~~~ #~~~
cfv_8000194_urllib                  = 7018668               #~~~ #~~~ #~~~
cfv_8000194_BeautifulSoup_alpha     = 'BeautifulSoup'       #~~~ #~~~ #~~~
cfv_8000194_datetime_alpha          = 'datetime'            #~~~ #~~~ #~~~
cfv_8000194_MySQLdb_alpha           = 'MySQLdb'             #~~~ #~~~ #~~~
cfv_8000194_numpy_alpha             = 'numpy'               #~~~ #~~~ #~~~
cfv_8000194_os_alpha                = 'os'                  #~~~ #~~~ #~~~
cfv_8000194_pprint_alpha            = 'pprint'              #~~~ #~~~ #~~~
cfv_8000194_requests_alpha          = 'requests'            #~~~ #~~~ #~~~
cfv_8000194_urllib_alpha            = 'urllib'              #~~~ #~~~ #~~~

edgarfetch_log_list.append('_____________________________________________________________________')

#
#~~~ ~~~~ 
#

print("Setting Classification Facet Keys and Class Facet Value Keys From Class Facet Copybook = EdgarFetch")
print("--------------------------------------------------------------------------------------------------- \n")
print("class_facet_8000850               = 8000850   #~~~ #~~~ Facet Key for: Dataset Content - Spoken Language of Content")
print("cfv_8000850_english                 = 7016098               #~~~ #~~~ #~~~")
print("cfv_8000850_english_alpha           = 'English'             #~~~ #~~~ #~~~")
print(" ")
print("class_facet_8000194               = 8000194   #~~~ #~~~ Facet Key for: Python Package Name")
print("cfv_8000194_BeautifulSoup           = 7018755               #~~~ #~~~ #~~~")
print("cfv_8000194_datetime                = 7018316               #~~~ #~~~ #~~~")
print("cfv_8000194_MySQLdb                 = 7018440               #~~~ #~~~ #~~~")
print("cfv_8000194_numpy                   = 7018460               #~~~ #~~~ #~~~")
print("cfv_8000194_os                      = 7018253               #~~~ #~~~ #~~~")
print("cfv_8000194_pprint                  = 7018501               #~~~ #~~~ #~~~")
print("cfv_8000194_requests                = 7018757               #~~~ #~~~ #~~~")
print("cfv_8000194_urllib                  = 7018668               #~~~ #~~~ #~~~")
print("cfv_8000194_BeautifulSoup_alpha     = 'BeautifulSoup'       #~~~ #~~~ #~~~")
print("cfv_8000194_datetime_alpha          = 'datetime'            #~~~ #~~~ #~~~")
print("cfv_8000194_MySQLdb_alpha           = 'MySQLdb'             #~~~ #~~~ #~~~")
print("cfv_8000194_numpy_alpha             = 'numpy'               #~~~ #~~~ #~~~")
print("cfv_8000194_os_alpha                = 'os'                  #~~~ #~~~ #~~~")
print("cfv_8000194_pprint_alpha            = 'pprint'              #~~~ #~~~ #~~~")
print("cfv_8000194_requests_alpha          = 'requests'            #~~~ #~~~ #~~~")
print("cfv_8000194_urllib_alpha            = 'urllib'              #~~~ #~~~ #~~~")
print(" ")
print("class_facet_8000830               = 8000830   #~~~ #~~~ Facet Key for: Dataset Storage Platform")
print("cfv_8000830_WindowsServer2012       = 7016289               #~~~ #~~~ #~~~")
print("cfv_8000830_WindowsServer2012_alpha = 'Windows Server 2012' #~~~ #~~~ #~~~")
print(" ")
print("class_facet_8000888               = 8000888   #~~~ #~~~ Facet Key for: Software Protocol")
print("class_facet_8000889               = 8000889   #~~~ #~~~ Facet Key for: Software Routine")
print("class_facet_8000890               = 8000890   #~~~ #~~~ Facet Key for: Job Log Prefix")
print(" ")
print("class_facet_8000857               = 8000857   #~~~ #~~~ Facet Key for: Hadoop/HDFS Cluster ID")
print("class_facet_8000861               = 8000861   #~~~ #~~~ Facet Key for: Hadoop HDFS Directory")
print("class_facet_8000862               = 8000862   #~~~ #~~~ Facet Key for: Windows Directory")
print("class_facet_8000870               = 8000870   #~~~ #~~~ Facet Key for: Storage Hierarchy Name")
print("class_facet_8000871               = 8000871   #~~~ #~~~ Facet Key for: local_server_os")
print("class_facet_8000872               = 8000872   #~~~ #~~~ Facet Key for: local_server_ip")
print("class_facet_8000887               = 8000887   #~~~ #~~~ Facet Key for: local_root_drive")
print("class_facet_8000873               = 8000873   #~~~ #~~~ Facet Key for: dir_purpose")
print("class_facet_8000874               = 8000874   #~~~ #~~~ Facet Key for: dir_source")
print("class_facet_8000875               = 8000875   #~~~ #~~~ Facet Key for: dir_source_subgroup_01")
print("class_facet_8000876               = 8000876   #~~~ #~~~ Facet Key for: dir_source_subgroup_02")
print("class_facet_8000877               = 8000877   #~~~ #~~~ Facet Key for: Job Log Suffix (used as Storage Hierarchy Node: JOB ID)")
print("class_facet_8000886               = 8000886   #~~~ #~~~ Facet Key for: Operating System Name")
print(" ")
print("class_facet_8000895               = 8000895   #~~~ #~~~ Facet Key for: MySQL host")
print("class_facet_8000896               = 8000896   #~~~ #~~~ Facet Key for: MySQL port")
print("class_facet_8000897               = 8000897   #~~~ #~~~ Facet Key for: MySQL userid")
print("class_facet_8000898               = 8000898   #~~~ #~~~ Facet Key for: MySQL database name")
print(" ")
print("class_facet_8000880               = 8000880   #~~~ #~~~ Facet Key for: EDGAR Filing: Filing Form")
print("class_facet_8000881               = 8000881   #~~~ #~~~ Facet Key for: EDGAR Filing: Accession Number")
print("class_facet_8000882               = 8000882   #~~~ #~~~ Facet Key for: EDGAR Filing: &type (i.e. filing type)")
print("class_facet_8000885               = 8000885   #~~~ #~~~ Facet Key for: EDGAR Filing: Central Index Key (CIK)")
print("class_facet_8000899               = 8000899   #~~~ #~~~ Facet Key for: EDGAR Filing: Filing Date")
print("class_facet_8000900               = 8000900   #~~~ #~~~ Facet Key for: EDGAR Filing: Accepted Date & Time")
print("class_facet_8000901               = 8000901   #~~~ #~~~ Facet Key for: EDGAR Filing: Documents")
print("class_facet_8000902               = 8000902   #~~~ #~~~ Facet Key for: EDGAR Filing: Period of Report")
print(" ")
print("class_facet_8000892               = 8000892   #~~~ #~~~ Facet Key for: targetDirectoryFtpDmz")
print("class_facet_8000893               = 8000893   #~~~ #~~~ Facet Key for: requestSentURL")
print("class_facet_8000894               = 8000894   #~~~ #~~~ Facet Key for: replyDocButtonURL")
print("class_facet_8000903               = 8000903   #~~~ #~~~ Facet Key for: Dataset Collection Name")
print("class_facet_8000905               = 8000905   #~~~ #~~~ Facet Key for: Dataset Description")

print("_____________________________________________________________________ \n")

#
#~~~ ~~~~
#

edgarfetch_log_list.append("Setting Classification Facet Keys and Class Facet Value Keys From Class Facet Copybook = EdgarFetch")
edgarfetch_log_list.append("-------------------------------------------------------------------------------------------------------------------")
edgarfetch_log_list.append(" ")
edgarfetch_log_list.append("class_facet_8000850               = 8000850   #~~~ #~~~ Facet Key for: Dataset Content - Spoken Language of Content")
edgarfetch_log_list.append("cfv_8000850_english                 = 7016098               #~~~ #~~~ #~~~")
edgarfetch_log_list.append("cfv_8000850_english_alpha           = 'English'             #~~~ #~~~ #~~~")
edgarfetch_log_list.append(" ")
edgarfetch_log_list.append("class_facet_8000194               = 8000194   #~~~ #~~~ Facet Key for: Python Package Name")
edgarfetch_log_list.append("cfv_8000194_BeautifulSoup           = 7018755               #~~~ #~~~ #~~~")
edgarfetch_log_list.append("cfv_8000194_datetime                = 7018316               #~~~ #~~~ #~~~")
edgarfetch_log_list.append("cfv_8000194_MySQLdb                 = 7018440               #~~~ #~~~ #~~~")
edgarfetch_log_list.append("cfv_8000194_numpy                   = 7018460               #~~~ #~~~ #~~~")
edgarfetch_log_list.append("cfv_8000194_os                      = 7018253               #~~~ #~~~ #~~~")
edgarfetch_log_list.append("cfv_8000194_pprint                  = 7018501               #~~~ #~~~ #~~~")
edgarfetch_log_list.append("cfv_8000194_requests                = 7018757               #~~~ #~~~ #~~~")
edgarfetch_log_list.append("cfv_8000194_urllib                  = 7018668               #~~~ #~~~ #~~~")
edgarfetch_log_list.append("cfv_8000194_BeautifulSoup_alpha     = 'BeautifulSoup'       #~~~ #~~~ #~~~")
edgarfetch_log_list.append("cfv_8000194_datetime_alpha          = 'datetime'            #~~~ #~~~ #~~~")
edgarfetch_log_list.append("cfv_8000194_MySQLdb_alpha           = 'MySQLdb'             #~~~ #~~~ #~~~")
edgarfetch_log_list.append("cfv_8000194_numpy_alpha             = 'numpy'               #~~~ #~~~ #~~~")
edgarfetch_log_list.append("cfv_8000194_os_alpha                = 'os'                  #~~~ #~~~ #~~~")
edgarfetch_log_list.append("cfv_8000194_pprint_alpha            = 'pprint'              #~~~ #~~~ #~~~")
edgarfetch_log_list.append("cfv_8000194_requests_alpha          = 'requests'            #~~~ #~~~ #~~~")
edgarfetch_log_list.append("cfv_8000194_urllib_alpha            = 'urllib'              #~~~ #~~~ #~~~")
edgarfetch_log_list.append(" ")
edgarfetch_log_list.append("class_facet_8000830               = 8000830   #~~~ #~~~ Facet Key for: Dataset Storage Platform")
edgarfetch_log_list.append("cfv_8000830_WindowsServer2012       = 7016289               #~~~ #~~~ #~~~")
edgarfetch_log_list.append("cfv_8000830_WindowsServer2012_alpha = 'Windows Server 2012' #~~~ #~~~ #~~~")
edgarfetch_log_list.append(" ")
edgarfetch_log_list.append("class_facet_8000857               = 8000857   #~~~ #~~~ Facet Key for: Hadoop/HDFS Cluster ID")
edgarfetch_log_list.append("class_facet_8000888               = 8000888   #~~~ #~~~ Facet Key for: Software Protocol")
edgarfetch_log_list.append("class_facet_8000889               = 8000889   #~~~ #~~~ Facet Key for: Software Routine")
edgarfetch_log_list.append("class_facet_8000890               = 8000890   #~~~ #~~~ Facet Key for: Job Log Prefix")
edgarfetch_log_list.append(" ")
edgarfetch_log_list.append("class_facet_8000861               = 8000861   #~~~ #~~~ Facet Key for: Hadoop HDFS Directory")
edgarfetch_log_list.append("class_facet_8000862               = 8000862   #~~~ #~~~ Facet Key for: Windows Directory")
edgarfetch_log_list.append("class_facet_8000870               = 8000870   #~~~ #~~~ Facet Key for: Storage Hierarchy Name")
edgarfetch_log_list.append("class_facet_8000871               = 8000871   #~~~ #~~~ Facet Key for: local_server_os")
edgarfetch_log_list.append("class_facet_8000872               = 8000872   #~~~ #~~~ Facet Key for: local_server_ip")
edgarfetch_log_list.append("class_facet_8000887               = 8000887   #~~~ #~~~ Facet Key for: local_root_drive")
edgarfetch_log_list.append("class_facet_8000873               = 8000873   #~~~ #~~~ Facet Key for: dir_purpose")
edgarfetch_log_list.append("class_facet_8000874               = 8000874   #~~~ #~~~ Facet Key for: dir_source")
edgarfetch_log_list.append("class_facet_8000875               = 8000875   #~~~ #~~~ Facet Key for: dir_source_subgroup_01")
edgarfetch_log_list.append("class_facet_8000876               = 8000876   #~~~ #~~~ Facet Key for: dir_source_subgroup_02")
edgarfetch_log_list.append("class_facet_8000877               = 8000877   #~~~ #~~~ Facet Key for: Job Log Suffix (used as Storage Hierarchy Node: JOB ID)")
edgarfetch_log_list.append("class_facet_8000886               = 8000886   #~~~ #~~~ Facet Key for: Operating System Name")
edgarfetch_log_list.append(" ")
edgarfetch_log_list.append("class_facet_8000895               = 8000895   #~~~ #~~~ Facet Key for: MySQL host")
edgarfetch_log_list.append("class_facet_8000896               = 8000896   #~~~ #~~~ Facet Key for: MySQL port")
edgarfetch_log_list.append("class_facet_8000897               = 8000897   #~~~ #~~~ Facet Key for: MySQL userid")
edgarfetch_log_list.append("class_facet_8000898               = 8000898   #~~~ #~~~ Facet Key for: MySQL database name")
edgarfetch_log_list.append(" ")
edgarfetch_log_list.append("class_facet_8000880               = 8000880   #~~~ #~~~ Facet Key for: EDGAR Filing: Filing Form")
edgarfetch_log_list.append("class_facet_8000881               = 8000881   #~~~ #~~~ Facet Key for: EDGAR Filing: Accession Number")
edgarfetch_log_list.append("class_facet_8000882               = 8000882   #~~~ #~~~ Facet Key for: EDGAR Filing: &type (i.e. filing type)")
edgarfetch_log_list.append("class_facet_8000885               = 8000885   #~~~ #~~~ Facet Key for: EDGAR Filing: Central Index Key (CIK)")
edgarfetch_log_list.append("class_facet_8000899               = 8000899   #~~~ #~~~ Facet Key for: EDGAR Filing: Filing Date")
edgarfetch_log_list.append("class_facet_8000900               = 8000900   #~~~ #~~~ Facet Key for: EDGAR Filing: Accepted Date & Time")
edgarfetch_log_list.append("class_facet_8000901               = 8000901   #~~~ #~~~ Facet Key for: EDGAR Filing: Documents")
edgarfetch_log_list.append("class_facet_8000902               = 8000902   #~~~ #~~~ Facet Key for: EDGAR Filing: Period of Report")
edgarfetch_log_list.append(" ")
edgarfetch_log_list.append("class_facet_8000892               = 8000892   #~~~ #~~~ Facet Key for: targetDirectoryFtpDmz")
edgarfetch_log_list.append("class_facet_8000893               = 8000893   #~~~ #~~~ Facet Key for: requestSentURL")
edgarfetch_log_list.append("class_facet_8000894               = 8000894   #~~~ #~~~ Facet Key for: replyDocButtonURL")
edgarfetch_log_list.append("class_facet_8000903               = 8000903   #~~~ #~~~ Facet Key for: Dataset Collection Name")
edgarfetch_log_list.append("class_facet_8000905               = 8000905   #~~~ #~~~ Facet Key for: Dataset Description")

edgarfetch_log_list.append("_____________________________________________________________________")
edgarfetch_log_list.append(" ")

#
#~~~ ~~~~ Python Routine Termination
#

print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>          Completed at: ' + str(datetime.datetime.now()))
print('_____________________________________________________________________')
print('_____________________________________________________________________')
print('_____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>          Completed at: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')

#
#~~~ ~~~~ MIT Creative Commons License
#
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------      Chameleon Metadata Python Software     -------------------------------------------
#------------------------------    © 2018 Chameleon Metadata (https://ChameleonMetadata.com)    -------------------------------- 
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------   MIT Creative Commons License   ------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------
# Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated  
# documentation files (the "Software"), to deal in the Software without restriction, including without limitation 
# the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and 
# to permit persons to whom the Software is furnished to do so, subject to the following conditions:
#
# 1) The above copyright notice & this permission notice shall be included in all copies or substantial portions of the Software.
#
# 2) The Software is provided "as is", without warranty of any kind, express or implied, including but not limited to 
#    the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors 
#    or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or 
#    otherwise, arising from, out of or in connection with the software or the use or other dealings in the Software.
#-------------------------------------------------------------------------------------------------------------------------------

>>>>>
>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S
>>>>>  _______________________________________________
>>>>>
>>>>>         R O U T I N E: EdgarFetch-V5.Load-EdgarFetch-Class-Facet-Copybook.18-12-10-12-50
>>>>>            Start Time: 2018-12-10 12:50:43.431880
>>>>> _____________________________________________________________________ 

Setting Classification Facet Keys and Class Facet Value Keys From Class Facet Copybook = EdgarFetch
--------------------------------------------------------------------------------------------------- 

class_facet_8000850               = 8000850   #~~~ #~~~ Facet Key for: Dataset Content - Spoken Language of Content
cfv_8000850_english                 = 7016098               #~~~ #~~~ #~~~
cfv_8000850_english_alpha           = 'English'             #~~~ #~~~ #~~~
 
class_facet_8000194               = 8000194   #~~~ #~~~ Facet Key for: Python Package Name
cfv_8000194_BeautifulSoup           = 7018755               #~~~ #~~~ #~~~
cfv_8000194_d

In [3]:
#~~~    Protocol
#~~~    EdgarFetch
#~~~ ~~~~    Routine
#~~~ ~~~~    Configure-and-Bind-Organization
#

job_log_prefix                 = 'Configure-and-Bind-Organization'
python_routine_long_name       = (python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)

print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>            Start Time: ' + str(datetime.datetime.now()))
print('_____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>        Routine Starts: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append(' ')


mysql_creator_org_connection = MySQLdb.Connection(host   = mysql_01_host, 
                                         port   = mysql_01_port,
                                         user   = mysql_01_user, 
                                         passwd = mysql_01_pw, 
                                         db     = mysql_01_db)

numpy_creator_org_config_bind_array = numpy.empty(shape=(0, 2),dtype='int32')

chameleon_creator_org_bind_list     = []

creator_org_bind_required           = 'N'

for (creator_org_master_key, sql_select_stmt, sql_insert_stmt) in numpy_creator_org_config_sql_array:

    print ('_____________________________________________________________________ \n')

    creator_org_master_key      = creator_org_master_key.replace('(', '')
    creator_org_master_key      = creator_org_master_key.replace(',', '')
    creator_org_master_key      = creator_org_master_key.replace(')', '')
    
    try:    
        dbhandler = mysql_creator_org_connection.cursor()
        dbhandler.execute(sql_select_stmt)
        existing_creator_org_results_set = dbhandler.fetchall()
        
        if not existing_creator_org_results_set:
            print('Organization Configuration Master Not Found: Creating It Now')
            
            try:    
                mysql_creator_org_handler =  mysql_creator_org_connection.cursor()
                mysql_creator_org_handler.execute(sql_insert_stmt)
                mysql_creator_org_connection.commit()
                print('Organization Configuration Master Record Added')
                creator_org_bind_required          = 'Y'
            
                try:
                    dbhandler = mysql_creator_org_connection.cursor()
                    dbhandler.execute(sql_select_stmt)
                    new_creator_org_results_set = dbhandler.fetchall()
                    for new_creator_org_result in new_creator_org_results_set:
                        print('Set To New Organization_config_master_key: ', str(new_creator_org_result))
                        creator_org_config_master_key       = str(new_creator_org_result)
                        
                except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                    print('MySQL Error - Organization Configuration Master Record Not Found After INSERT')
                    print(MySQLexception, MySQLdb.Warning)
                    mysql_creator_org_connection.rollback()
            
            except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                    print('MySQL Error - Organization Configuration Master Record INSERT Attempted & Failed')
                    print(MySQLexception, MySQLdb.Warning)
                    mysql_creator_org_connection.rollback()           
        else:    
            for existing_creator_org_result in existing_creator_org_results_set:
                print('Organization Configuration Master Already Exists: ')
                print('Organization_config_master_key Set To: ', str(existing_creator_org_result))
                creator_org_config_master_key       = str(existing_creator_org_result)
                
    except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
        print('MySQL Error - Organization Configuration Master Lookup Failure - Initial SELECT Statement')
        print(MySQLexception, MySQLdb.Warning)
    
    creator_org_config_master_key      = creator_org_config_master_key.replace('(', '')
    creator_org_config_master_key      = creator_org_config_master_key.replace(',', '')
    creator_org_config_master_key      = creator_org_config_master_key.replace(')', '')
    
    creator_org_config_bind_array      = numpy.array([int(source_org_config_master_key), int(creator_org_config_master_key)])
    
    if creator_org_config_bind_array not in numpy_creator_org_config_bind_array:
        numpy_creator_org_config_bind_array = numpy.append(numpy_creator_org_config_bind_array, [creator_org_config_bind_array], axis=0) 

    org_auto_bind_notes         = '(Org:' + str(creator_org_master_key) + ') Auto-Bind to (Org Config:' + str(creator_org_config_master_key) + ') Done By: ' + python_protocol + '_' + job_log_suffix 
    org_auto_bind_insert_string = ('INSERT INTO organization_config_bind VALUES (NULL , ' + str(creator_org_master_key) + ' , '  + str(creator_org_config_master_key) +  ' , 0, 0, CURRENT_TIMESTAMP, NULL, NULL, ' + "'" + org_auto_bind_notes + "'" + ')')

    chameleon_creator_org_bind_list.append(org_auto_bind_insert_string)

for chameleon_creator_org_bind_item in chameleon_creator_org_bind_list: 
    if creator_org_bind_required         == 'Y':
        try:    
            mysql_insert_creator_org_handler =  mysql_creator_org_connection.cursor()
            mysql_insert_creator_org_handler.execute(org_auto_bind_insert_string)
            mysql_creator_org_connection.commit()
            print('Organization Bind (Org Master:', str(creator_org_master_key), ') << Now Bound To >> (Org Config: ', str(creator_org_config_master_key) ,')')
            creator_org_bind_required          = 'N'
            
        except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
            print('MySQL Error - Organization Configuration Bind Attempted for: (' + str(creator_org_master_key) + '<< BIND TO >>' + str(creator_org_config_master_key) + ') Attempted & Failed')
            print(MySQLexception, MySQLdb.Warning)
            mysql_creator_org_connection.rollback()

mysql_creator_org_connection.close()

edgarfetch_log_list.append(' \n')
edgarfetch_log_list.append('numpy_creator_org_config_bind_array')
edgarfetch_log_list.append('([[ source_org_cmk, creator_org_master_key,  creator_org_cmk]]) \n')
for source_creator_org_config_combo in numpy_creator_org_config_bind_array:
    edgarfetch_log_list.append(source_creator_org_config_combo)


print('_____________________________________________________________________ \n')

   
#
#~~~ ~~~~ Python Routine Termination
#

print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>          Completed at: ' + str(datetime.datetime.now()))
print('_____________________________________________________________________')
print('_____________________________________________________________________')
print('_____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>          Completed at: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')

#
#~~~ ~~~~ MIT Creative Commons License
#
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------      Chameleon Metadata Python Software     -------------------------------------------
#------------------------------    © 2018 Chameleon Metadata (https://ChameleonMetadata.com)    -------------------------------- 
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------   MIT Creative Commons License   ------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------
# Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated  
# documentation files (the "Software"), to deal in the Software without restriction, including without limitation 
# the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and 
# to permit persons to whom the Software is furnished to do so, subject to the following conditions:
#
# 1) The above copyright notice & this permission notice shall be included in all copies or substantial portions of the Software.
#
# 2) The Software is provided "as is", without warranty of any kind, express or implied, including but not limited to 
#    the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors 
#    or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or 
#    otherwise, arising from, out of or in connection with the software or the use or other dealings in the Software.
#-------------------------------------------------------------------------------------------------------------------------------

>>>>>
>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S
>>>>>  _______________________________________________
>>>>>
>>>>>         R O U T I N E: EdgarFetch-V5.Configure-and-Bind-Organization.18-12-10-12-50
>>>>>            Start Time: 2018-12-10 12:50:51.898002
_____________________________________________________________________ 

_____________________________________________________________________ 

Organization Configuration Master Already Exists: 
Organization_config_master_key Set To:  (3300087,)
_____________________________________________________________________ 

>>>>>
>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D
>>>>>  _______________________________________________
>>>>>
>>>>>         R O U T I N E: EdgarFetch-V5.Configure-and-Bind-Organization.18-12-10-12-50
>>>>>          Completed at: 2018-12-10 12:50:52.396712
_____________________________________________________________________
_____________________________________________________________________


In [4]:
#~~~    Protocol
#~~~    EdgarFetch
#~~~ ~~~~    Routine
#~~~ ~~~~    Configure-and-Bind-Routine
#

job_log_prefix                 = 'Configure-and-Bind-Routine'
python_routine_long_name       = (python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)

print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>            Start Time: ' + str(datetime.datetime.now()))
print('_____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>        Routine Starts: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append(' ')


mysql_routine_connection = MySQLdb.Connection(host   = mysql_01_host, 
                                              port   = mysql_01_port,
                                              user   = mysql_01_user, 
                                              passwd = mysql_01_pw, 
                                              db     = mysql_01_db)

numpy_routine_config_bind_array = numpy.empty(shape=(0, 2),dtype='int32')

routine_bind_list               = []

routine_bind_required           = 'N'

for (routine_master_key, sql_select_stmt, sql_insert_stmt) in numpy_routine_config_sql_array:

    print('routine_master_key: ', routine_master_key, ' \n')

    routine_master_key      = routine_master_key.replace('(', '')
    routine_master_key      = routine_master_key.replace(',', '')
    routine_master_key      = routine_master_key.replace(')', '')
    
    try:    
        dbhandler = mysql_routine_connection.cursor()
        dbhandler.execute(sql_select_stmt)
        existing_routine_results_set = dbhandler.fetchall()
    
        if not existing_routine_results_set:
            print('Routine Configuration Master Not Found: Creating It Now')
            
            try:    
                mysql_routine_handler =  mysql_routine_connection.cursor()
                mysql_routine_handler.execute(sql_insert_stmt)
                mysql_routine_connection.commit()
                print('Routine Configuration Master Record Added')
                routine_bind_required          = 'Y'
            
                try:
                    dbhandler = mysql_routine_connection.cursor()
                    dbhandler.execute(sql_select_stmt)
                    new_routine_results_set = dbhandler.fetchall()
                    for new_routine_result in new_routine_results_set:
                        print('routine_config_master_key Set To: ', str(new_routine_result))
                        routine_config_master_key       = str(new_routine_result)
                        
                except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                    print('MySQL Error - Routine Configuration Master Record Not Found After INSERT')
                    print(MySQLexception, MySQLdb.Warning)
                    mysql_routine_connection.rollback()
            
            except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                    print('MySQL Error - Routine Configuration Master Record INSERT Attempted & Failed')
                    print(MySQLexception, MySQLdb.Warning)
                    mysql_routine_connection.rollback()           
        else:    
            for existing_routine_result in existing_routine_results_set:
                print('Routine Configuration Master Already Exists: ')
                print('routine_config_master_key Set To: ', str(existing_routine_result))
                routine_config_master_key       = str(existing_routine_result)
                
    except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
        print('MySQL Error - Routine Configuration Master Lookup Failure - Initial SELECT Statement')
        print(MySQLexception, MySQLdb.Warning)
    
    routine_config_master_key      = routine_config_master_key.replace('(', '')
    routine_config_master_key      = routine_config_master_key.replace(',', '')
    routine_config_master_key      = routine_config_master_key.replace(')', '')
    
    routine_config_array               = numpy.array([int(routine_master_key), int(routine_config_master_key)])
    numpy_routine_config_bind_array    = numpy.append(numpy_routine_config_bind_array, [routine_config_array], axis=0) 


    routine_auto_bind_notes         = '(Routine: ' + str(routine_master_key) + ') Auto-Bind to (Routine Config: ' + str(routine_config_master_key) + ') Done By: ' + python_protocol + '_' + job_log_suffix 
    routine_auto_bind_insert_string = ('INSERT INTO routine_config_bind VALUES (NULL , ' + str(routine_master_key) + ' , '  + str(routine_config_master_key) +  ' , 0, 0, CURRENT_TIMESTAMP, NULL, NULL, ' + "'" + routine_auto_bind_notes + "'" + ')')

    routine_bind_list.append(routine_auto_bind_insert_string)

for routine_bind_item in routine_bind_list: 
    if routine_bind_required         == 'Y':
        try:    
            mysql_insert_routine_handler =  mysql_routine_connection.cursor()
            mysql_insert_routine_handler.execute(routine_auto_bind_insert_string)
            mysql_routine_connection.commit()
            print('Routine Bind (Routine Master:', str(routine_master_key), ') << Now Bound To >> (Routine Config: ', str(routine_config_master_key) ,')')
            routine_bind_required          = 'N'
            
        except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
            print('MySQL Error - Routine Configuration Bind Attempted for: (' + str(routine_master_key) + '<< BIND TO >>' + str(routine_config_master_key) + ') Attempted & Failed')
            print(MySQLexception, MySQLdb.Warning)
            mysql_routine_connection.rollback()

mysql_routine_connection.close()    

edgarfetch_log_list.append(' \n')
edgarfetch_log_list.append('numpy_routine_config_bind_array')
edgarfetch_log_list.append('([[ routine_master_key,  routine_config_master_key]]) \n')
edgarfetch_log_list.append(numpy_routine_config_bind_array)

edgarfetch_log_list.append('_____________________________________________________________________ \n')

print(' \n')
print('numpy_routine_config_bind_array')
print('([[ routine_master_key,  routine_config_master_key]]) \n')
pprint.pprint(numpy_routine_config_bind_array)

print('_____________________________________________________________________ \n')
    
#
#~~~ ~~~~ Python Routine Termination
#

print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>          Completed at: ' + str(datetime.datetime.now()))
print('_____________________________________________________________________')
print('_____________________________________________________________________')
print('_____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>          Completed at: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')

#
#~~~ ~~~~ MIT Creative Commons License
#
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------      Chameleon Metadata Python Software     -------------------------------------------
#------------------------------    © 2018 Chameleon Metadata (https://ChameleonMetadata.com)    -------------------------------- 
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------   MIT Creative Commons License   ------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------
# Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated  
# documentation files (the "Software"), to deal in the Software without restriction, including without limitation 
# the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and 
# to permit persons to whom the Software is furnished to do so, subject to the following conditions:
#
# 1) The above copyright notice & this permission notice shall be included in all copies or substantial portions of the Software.
#
# 2) The Software is provided "as is", without warranty of any kind, express or implied, including but not limited to 
#    the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors 
#    or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or 
#    otherwise, arising from, out of or in connection with the software or the use or other dealings in the Software.
#-------------------------------------------------------------------------------------------------------------------------------

>>>>>
>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S
>>>>>  _______________________________________________
>>>>>
>>>>>         R O U T I N E: EdgarFetch-V5.Configure-and-Bind-Routine.18-12-10-12-50
>>>>>            Start Time: 2018-12-10 12:51:02.540959
_____________________________________________________________________ 

routine_master_key:  40000001  

Routine Configuration Master Not Found: Creating It Now
Routine Configuration Master Record Added
routine_config_master_key Set To:  (1,)
Routine Bind (Routine Master: 40000001 ) << Now Bound To >> (Routine Config:  1 )
 

numpy_routine_config_bind_array
([[ routine_master_key,  routine_config_master_key]]) 

array([[40000001,        1]])
_____________________________________________________________________ 

>>>>>
>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D
>>>>>  _______________________________________________
>>>>>
>>>>>         R O U T I N E: EdgarFetch-V5.Configure-and-Bind-Routine.18-12-10-12-50
>>>>> 

In [5]:
#~~~    Protocol
#~~~    EdgarFetch
#~~~ ~~~~    Routine
#~~~ ~~~~    Receive-Native-Datasets-into-DMZ-Server
#

job_log_prefix                 = 'Receive-Native-Datasets-into-DMZ-Server'
python_routine_long_name       = (python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
                           
print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>            Start Time: ' + str(datetime.datetime.now()))
print('>>>>>  _____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>        Routine Starts: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append(' ')


#~~~ ~~~~ NOTICE: Unique MySQL Alias reduces returning bad working storage memory location from previous MySQL queries

mysql_dmz_dataset_connection = MySQLdb.Connection(host   = mysql_01_host, 
                                                  port   = mysql_01_port,
                                                  user   = mysql_01_user, 
                                                  passwd = mysql_01_pw, 
                                                  db     = mysql_01_db)

#~~~ ~~~~

sec_edgar_reply_metadata_list               = []
sec_edgar_doc_infoHeads_list                = []
sec_edgar_doc_infos_list                    = []
sec_edgar_document_url_list                 = []
sec_edgar_document_metadata_list            = []

hadoop_mkdir_command_list                   = []
hadoop_copy_command_list                    = []

dmz_dataset_registration_sql_list           = []
numpy_dmz_dataset_registraton_sql_array     = numpy.empty((0, 7))

numpy_new_dmz_dataset_array                 = numpy.empty(shape=(0, 6))
numpy_new_dmz_dataset_config_array          = numpy.empty(shape=(0, 3))
numpy_new_dmz_dataset_config_grain_array    = numpy.empty(shape=(0, 3))

org_facet_value_sql_list             = []
numpy_org_facet_value_sql_array      = numpy.empty((0, 3))

routine_facet_value_sql_list         = []
numpy_routine_facet_value_sql_array  = numpy.empty((0, 3))

dataset_facet_value_sql_list         = []
numpy_dataset_facet_value_sql_array  = numpy.empty((0, 3))


#~~~ ~~~~

wss_dataset_collection_name                 = ' '
wss_is_org_config_grain                     = 'N'
wss_is_routine_config_grain                 = 'N'
wss_is_dataset_config_grain                 = 'N'
wss_sql_statement_class_facet               = 0
wss_sql_statement_cfv_value_alpha           = ' '

#~~~ ~~~~ Open Edgar Reply URL's - one for each Documents Button (i.e. set of filings like a 10-K)

for sec_edgar_doc_button_url in sec_edgar_doc_buttons_url_list:
    with urllib.request.urlopen(sec_edgar_doc_button_url) as edgar_request_url:
        edgar_request_url_read = edgar_request_url.read()

#~~~ ~~~~ 2.2.3 Initialize 'Loop BeautifulSoups' for each new Edgar Reply URL 


    soup_edgar_request              = BeautifulSoup(edgar_request_url_read, "html.parser")
    
    soup_edgar_companyInfo          = soup_edgar_request.find_all('div', {"class": "companyInfo"})
    soup_edgar_companyNames         = soup_edgar_request.find_all('span', {"class": "companyName"})
    soup_edgar_secNums              = soup_edgar_request.find_all('div', id='secNum')
    soup_edgar_formNames            = soup_edgar_request.find_all('div', id='formName')
    soup_edgar_infoHeads            = soup_edgar_request.find_all('div', {"class": "infoHead"})
    soup_edgar_infos                = soup_edgar_request.find_all('div', {"class": "info"})
    soup_document_format_files      = soup_edgar_request.find_all('div', {"summary": "Document Format Files"})
    soup_edgar_request_tables       = soup_edgar_request.find_all('table')

#~~~ ~~~~ 2.2.4 Capture Company Name

    
    for edgar_companyName in soup_edgar_companyNames:
        edgar_companyName_text      = edgar_companyName.text
        edgar_companyName_text      = edgar_companyName_text.replace('\n', '') 
        edgar_companyName_text      = edgar_companyName_text.replace(' (see all company filings)', '')            
            

#~~~ ~~~~ 2.2.7 Capture SEC Accession Number for this Filing's Documents Page

            
    for edgar_secNum in soup_edgar_secNums:   
        edgar_secNum                      = edgar_secNum
        edgar_secNum_only                 = (edgar_secNum.text[19:])
        
        edgar_secNum_only                 = edgar_secNum_only.replace(' ', '')
        edgar_secNum_only                 = edgar_secNum_only.replace('\n', '')
        
        edgar_secNum_clean                = edgar_secNum_only.replace('-', '')

        wss_dataset_collection_name       = ('EDGAR.' 
                                             + str(sec_company_cik) + '.' 
                                             + str(edgar_secNum_only))
              
#~~~ ~~~~ 2.2.8 Build Output Directory Names for each Filing's Documents Page


        output_os_dir_source_subgroup_01_job = (output_os_dir_purpose_src_prefix + '/'
                                                + sec_company_cik + '/'
                                                + edgar_secNum_only)
    
        output_os_dir_source_subgroup_02_job = (output_os_dir_purpose_src_prefix + '/'
                                                + sec_company_cik + '/'
                                                + edgar_secNum_only + '/'
                                                + job_log_suffix)
        
        hadoop_target_dir_JOB_ID             = (hadoop_purpose_source1_prefix + '/'
                                                + sec_company_cik + '/'
                                                + edgar_secNum_only + '/'
                                                + job_log_suffix)
        
        hadoop_target_dir_slash_secNum       = ('/' + hadoop_purpose_source1_prefix + '/'
                                                + sec_company_cik + '/'
                                                + edgar_secNum_only)

                                     
        hadoop_mkdir_source_subgroup_02      = (hadoop_mkdir_command_prefix + '/'
                                                + dir_purpose + '/'
                                                + dir_source
                                                + sec_company_cik + '/'
                                                + edgar_secNum_only + '/'
                                                + job_log_suffix)
        
        hadoop_copy_dir_source_subgroup_02_job       = (hadoop_copyFromLocal_prefix + '/'
                                             + sec_company_cik + '/'
                                             + edgar_secNum_only + '/'
                                             + job_log_suffix +'"' + ' ') #~~~ Don't forget the trailing blank space             


#~~~ ~~~~ 2.2.9 Confirm or Create Output Directories for each Filing's Documents Page

        print('_____________________________________________________________________ \n')
    
        edgarfetch_log_list.append('_____________________________________________________________________')
        edgarfetch_log_list.append(' ')

        if not os.path.exists(output_os_dir_source_subgroup_02_job):
            print('Output Directory Not Found:   ', output_os_dir_source_subgroup_02_job)
            edgarfetch_log_list.append('Output Directory Not Found:')
            edgarfetch_log_list.append(' ~ ' + output_os_dir_source_subgroup_02_job)
            edgarfetch_log_list.append(' ~ ')
            try:
                os.makedirs(output_os_dir_source_subgroup_02_job)
                print(' ~ Output Directory Created:  ', output_os_dir_source_subgroup_02_job)
                edgarfetch_log_list.append(' ~ Output Directory Created: ')
                edgarfetch_log_list.append(' ~ ~ ' + output_os_dir_source_subgroup_02_job)
            except:
                print(' ~ ERROR: Creating Output Directory')
                edgarfetch_log_list.append(' ~ ~ ERROR: Creating Output Directory' + ' = ' + output_os_dir_source_subgroup_02_job)
        else:
            print('Output Directory Already Exists: ', output_os_dir_source_subgroup_02_job)
            edgarfetch_log_list.append('Output Directory Already Exists: ')
            edgarfetch_log_list.append(' ~ ' + output_os_dir_source_subgroup_02_job)
            

#~~~ ~~~~ Capture Documents Page Summary Text (formName = 10-K, 13F, etc.) and Heading Tags (soup_edgar_infoHeads = Date Filed, Number of Pages etc.) and  

    for edgar_formName in soup_edgar_formNames:
        edgar_formName_only           = (edgar_formName.text)
        edgar_formName_only           = edgar_formName_only.replace('\n', '')

    for edgar_infoHead in soup_edgar_infoHeads:
        edgar_infoHead_only           = (edgar_infoHead.text)

        edgar_infoHead_only           = edgar_infoHead_only.replace('\n', '')
        
        sec_edgar_doc_infoHeads_list.append(edgar_infoHead_only)
            
#~~~ ~~~~ Capture Documents Page Heading Values (soup_edgar_infos) for each infoHead (i.e. Number of Pages = 97)

    for edgar_info in soup_edgar_infos:
        sec_edgar_doc_infos_list.append(edgar_info.text)
            
#~~~ ~~~~ Link the Heading Tags to their associated Heading Values

    for i_infoHead, enumerated_doc_infoHead in enumerate(sec_edgar_doc_infoHeads_list):
        enumerated_edgar_info           = sec_edgar_doc_infos_list[i_infoHead] 

#~~~ ~~~~ Capture 'Page Metadata' from each Filing's Documents Page (Not the datasets included in the filing)

    wss_edgar_filing_date               = sec_edgar_doc_infos_list[0]
    wss_edgar_filing_accepted           = sec_edgar_doc_infos_list[1]
    wss_edgar_filing_documents          = sec_edgar_doc_infos_list[2]
    wss_edgar_filing_period_of_rpt      = sec_edgar_doc_infos_list[3]
    
    print('wss_edgar_filing_date               =  ', wss_edgar_filing_date)
    print('wss_edgar_filing_accepted           =  ', wss_edgar_filing_accepted)
    print('wss_edgar_filing_documents          =  ', wss_edgar_filing_documents)
    print('wss_edgar_filing_period_of_rpt      =  ', wss_edgar_filing_period_of_rpt)

    edgar_reply_metadata_item           = sec_company_cik + ', ' 
    edgar_reply_metadata_item           = edgar_reply_metadata_item + edgar_companyName_text + ', ' 
    edgar_reply_metadata_item           = edgar_reply_metadata_item + edgar_secNum_only + ', ' 
    edgar_reply_metadata_item           = edgar_reply_metadata_item + edgar_secNum_clean + ', ' 
    edgar_reply_metadata_item           = edgar_reply_metadata_item + edgar_formName_only + ', ' 
    edgar_reply_metadata_item           = edgar_reply_metadata_item + wss_edgar_filing_date  + ', ' 
    edgar_reply_metadata_item           = edgar_reply_metadata_item + wss_edgar_filing_accepted + ', ' 
    edgar_reply_metadata_item           = edgar_reply_metadata_item + wss_edgar_filing_documents  + ', ' 
    edgar_reply_metadata_item           = edgar_reply_metadata_item + wss_edgar_filing_period_of_rpt  + ', ' 
    edgar_reply_metadata_item           = edgar_reply_metadata_item + python_protocol + ', '
    edgar_reply_metadata_item           = edgar_reply_metadata_item + job_log_prefix + ', '
    edgar_reply_metadata_item           = edgar_reply_metadata_item + job_log_suffix + ', '
    edgar_reply_metadata_item           = edgar_reply_metadata_item + storage_hierarchy_name + ','
    edgar_reply_metadata_item           = edgar_reply_metadata_item + output_os_dir_source_subgroup_02_job + ', '
    edgar_reply_metadata_item           = edgar_reply_metadata_item + output_os_replyDocButtons_file + ', '
    edgar_reply_metadata_item           = edgar_reply_metadata_item + hadoop_target_dir_JOB_ID + ', '
    edgar_reply_metadata_item           = edgar_reply_metadata_item + hadoop_mkdir_source_subgroup_02 + ', '
    edgar_reply_metadata_item           = edgar_reply_metadata_item + sec_edgar_doc_button_url

    sec_edgar_reply_metadata_list.append(edgar_reply_metadata_item)
        
    print(' ~ ~ Writing Edgar Filing Datasets to Target Directory for SEC Accession Number: ' + edgar_secNum_only)
     
    edgarfetch_log_list.append(' ~')
    edgarfetch_log_list.append(' ~ Fetching All Datasets Included With SEC Accession Number: ' + edgar_secNum_only)
    edgarfetch_log_list.append(' ')
    
#~~~ ~~~~ Build the Hadoop (mkdir) Commands for each Documents Page (i.e. each SEC Filing)

    if hadoop_mkdir_source_subgroup_02 not in hadoop_mkdir_command_list:    
        hadoop_mkdir_command_list.append(hadoop_mkdir_source_subgroup_02)

#~~~ Now, parse the HTML table on each 'Documents Button' page for metadata about each of the many datasets per form (per 10-K, etc.)

#~~~ HEADS UP: This bit is a little tricky.  We have a table with 5 columns and an unknown number of rows.
#~~~           But, since we know we have 5 columns, we also know each time the counter reaches 6, it means 
#~~~           a new table row & document.  
#~~~           So, when the counter reaches 5, we store the document metadata, build '.metadata' file names, 
#~~~           add to Python dictionaries and write the actual file to the directory whose name was pre-built 
#~~~           based on the HTML details collected for each dataset before the WRITE occurs


#~~~ ~~~~ 2.2.16 Capture List of each Document and its Metadata


    for edgar_request_table in soup_edgar_request_tables:
        edgar_form_page_metadata_count             = 0

        for td in edgar_request_table.findAll("td"):            
            edgar_form_page_metadata_count = edgar_form_page_metadata_count + 1

            if edgar_form_page_metadata_count      >= 6:
                edgar_form_page_metadata_count     = 1                
                td_findNext_string                 = (td.findNext(text=True))
            else:
                td_findNext_string                 = (td.findNext(text=True))
                
                print(edgar_form_page_metadata_count, ': ',td_findNext_string)  
    
                if (edgar_form_page_metadata_count == 2):               #~~~ 2 is Document Description 
                    metadata_document_description  = td_findNext_string    
                
                if (edgar_form_page_metadata_count == 3):               #~~~ 3 is Document Name                
                    dmz_dataset_name_raw = td_findNext_string
                    dmz_dataset_name_raw = dmz_dataset_name_raw.replace('\n', '')
            

                if (edgar_form_page_metadata_count == 5):               #~~~ 5 is Document Size

                    
                    wss_edgar_filing_date             = sec_edgar_doc_infos_list[0]
                    wss_edgar_filing_accepted         = sec_edgar_doc_infos_list[1]
                    wss_edgar_filing_documents        = sec_edgar_doc_infos_list[2]
                    wss_edgar_filing_period_of_rpt    = sec_edgar_doc_infos_list[3]
                    
                    metadata_document_size = td_findNext_string 
                    
                    wss_hadoop_copy_command     = (hadoop_copy_dir_source_subgroup_02_job + hadoop_target_dir_JOB_ID)

#~~~ ~~~~ 2.2.17 Build Hadoop (copyFromLocal) Command
                                        
                    if  wss_hadoop_copy_command not in hadoop_copy_command_list:
                        hadoop_copy_command_list.append(wss_hadoop_copy_command)

#~~~ ~~~~ 2.2.18 Build and Capture Metadata Record for each Document
  
                    dmz_dataset_name_tagged      = (dmz_dataset_name_raw + '_' 
                                                    + sec_company_cik + '_'
                                                    + edgar_secNum_only + '_' 
                                                    + job_log_suffix) 

# 
#~~~ ~~~~ Capture the Metadata Inherited by ALL Datasets in each SEC Filing (Accession Number) in the Reply from Edgar
#
          
                    for (source_org_config_master_key, creator_org_config_master_key) in numpy_creator_org_config_bind_array:
        
                        dmz_document_metadata    = (sec_company_cik  + ', '
                                                    + metadata_document_description + ', '
                                                    + dmz_dataset_name_raw + ', '
                                                    + dmz_dataset_name_tagged + ', '
                                                    + edgar_secNum_only  + ', '   
                                                    + python_protocol + ', '
                                                    + job_log_suffix + ', '
                                                    + output_os_directory_cik_prefix + ', '
                                                    + metadata_document_size + ', '
                                                    + str(source_org_config_master_key) + ', '
                                                    + str(creator_org_config_master_key) + ', '
                                                    + str(record_content_type_key) + ', '
                                                    + sec_edgar_doc_button_url)

                        if dmz_document_metadata not in sec_edgar_document_metadata_list:
                            sec_edgar_document_metadata_list.append(dmz_document_metadata)


#
#~~~ ~~~~ 2.2.19 Build Directory and File Names for each Filing Dataset
#
                
                        output_os_replyDocsMetadata_file  = (output_os_directory_cik_prefix + '/'
                                                             + python_protocol + '_'
                                                             + job_log_suffix + '_replyDocuments.metadata')
                    
                        output_os_replyDocsURL_file       = (output_os_directory_cik_prefix + '/'
                                                             + python_protocol + '_'
                                                             + job_log_suffix + '_datasetSourceURL.reply')
                    
                        output_os_hadoop_copy_cmds_file   = (output_os_directory_cik_prefix + '/'
                                                             + python_protocol + '_'
                                                             + job_log_suffix + '_copyFromLocal.hadoop')
    
                        sec_edgar_document_request_url    = (sec_edgar_request_url_prefix + '/'
                                                             + sec_edgar_document_request_prefix + '/'
                                                             + sec_company_cik + '/'
                                                             + edgar_secNum_clean + '/'
                                                             + dmz_dataset_name_raw)
            
                        output_os_replyDocument_file      = (output_os_dir_source_subgroup_02_job + '/'
                                                             + dmz_dataset_name_raw)

                        dmz_dataset_local_dmz_url         = ('file:' + '//'
                                                             + windows_server_name + '/'
                                                             + local_root_drive + '/'
                                                             + dir_purpose + '/'
                                                             + dir_source + '/'
                                                             + sec_company_cik + '/'
                                                             + edgar_secNum_only + '/'
                                                             + job_log_suffix + '/'
                                                             + dmz_dataset_name_raw)
# 
#~~~ ~~~~ Register EACH DIFFERENT DMZ DATASET included in each SEC Filing (Accession Number) in the Reply from Edgar
#                                   

                    if sec_edgar_document_request_url not in sec_edgar_document_url_list:
                        sec_edgar_document_url_list.append(sec_edgar_document_request_url)
                                             
                        requestDocURL  = requests.get(sec_edgar_document_request_url, allow_redirects=True)
                        
                        with open(output_os_replyDocument_file, 'wb')as docfile_request:
                            docfile_request.write(requestDocURL.content)
                            
                            docfile_request.close()
                            
                            print(' ~ ~ Onboarding Dataset: ' + output_os_replyDocument_file)
                            edgarfetch_log_list.append(' ~ ~ Onboarding Dataset: ' + output_os_replyDocument_file)

                            dataset_config_notes            = (dmz_dataset_name_raw 
                                                            + ': Onboarded into the DMZ by <'
                                                            + python_protocol + '_' 
                                                            + job_log_suffix  + '> as: ' 
                                                            + dmz_dataset_name_tagged 
                                                            +' with Source Org Config = ' 
                                                            + str(source_org_config_master_key) 
                                                            + ' & Creator Org Config = ' 
                                                            + str(creator_org_config_master_key))
    
                            dataset_config_select_string    = ('SELECT dataset_config_master_key FROM dataset_config_master WHERE dataset_config_master_name = ' + "'" + dmz_dataset_name_tagged + "'")        
                            dataset_config_insert_string    = ('INSERT INTO dataset_config_master VALUES (NULL' + ', ' + str(57) + ', ' + '0, 0, NULL, CURRENT_TIMESTAMP, NULL, NULL' + ', ' + "'" + dmz_dataset_name_tagged + "'" + ', ' + "'" + dataset_config_notes + "'" + ')')

                            dataset_config_sql_string       = (dmz_dataset_name_raw + ', ' + dmz_dataset_name_tagged + ', ' + str(creator_org_config_master_key) + ' , ' + str(source_org_config_master_key) + ' , ' + ' , ' + dataset_config_select_string + ' , ' + dataset_config_insert_string)
                            dataset_config_sql_array        = numpy.array([str(creator_org_config_master_key), str(source_org_config_master_key), dmz_dataset_name_raw, dmz_dataset_name_tagged, dataset_config_select_string , dataset_config_insert_string])
#
#~~~ ~~~~
#

                            for (routine_master_key, routine_config_master_key) in numpy_routine_config_bind_array:

                                dmz_dataset_registraton_select_string = ('SELECT dmz_dataset_key FROM dmz_dataset WHERE dmz_dataset_name_tagged = ' 
                                                                         + "'" + dmz_dataset_name_tagged + "'" )
            
                                dmz_dataset_registraton_insert_string = ('INSERT INTO dmz_dataset VALUES (NULL' + ', ' 
                                                                            + str(creator_org_config_master_key) + ' , ' 
                                                                            + str(source_org_config_master_key) + ' , ' 
                                                                            + str(routine_config_master_key) + ' , ' 
                                                                            + str(2) + ', '  ## Dataset Source Type 2 = External
                                                                            + str(53) + ', ' ## Data Steward
                                                                            + 'NULL, CURRENT_TIMESTAMP, NULL' + ', ' 
                                                                            + "'" + dmz_dataset_name_tagged + "'" + ', ' 
                                                                            + "'" + dmz_dataset_name_raw + "'" + ', ' 
                                                                            + "'" + dataset_config_notes + "'" + ', ' 
                                                                            + "'" + sec_edgar_document_request_url + "'" + ', '
                                                                            + "'" + dmz_dataset_local_dmz_url + "'" + ', '
                                                                            + 'NULL)')

                                dmz_dataset_registraton_sql_string       = (dmz_dataset_name_raw + ', ' + dmz_dataset_name_tagged + ', ' + str(creator_org_config_master_key) + ' , ' + str(source_org_config_master_key) + ' , ' + ' , ' + dataset_config_select_string + ' , ' + dataset_config_insert_string)
                                dmz_dataset_registraton_sql_array        = numpy.array([str(creator_org_config_master_key), str(source_org_config_master_key), str(routine_config_master_key), dmz_dataset_name_raw, dmz_dataset_name_tagged, dmz_dataset_registraton_select_string , dmz_dataset_registraton_insert_string])

                                if dmz_dataset_registraton_sql_string not in dmz_dataset_registration_sql_list:
                                    dmz_dataset_registration_sql_list.append(dmz_dataset_registraton_sql_string)
                                    numpy_dmz_dataset_registraton_sql_array = numpy.append(numpy_dmz_dataset_registraton_sql_array, [dmz_dataset_registraton_sql_array], axis=0)




#
# ~~~ ~~~~ REGISTER DMZ DATASET     INTO CHAMELEON TABLE: dmz_dataset
#
                                    print('Registering Dataset in DMZ with Name: ', dmz_dataset_name_tagged)
                                    
                                    try:    
                                        dbhandler = mysql_dmz_dataset_connection.cursor()
                                        dbhandler.execute(dmz_dataset_registraton_select_string)
                                        existing_dmz_dataset_results_set = dbhandler.fetchall()
                                    
                                        if not existing_dmz_dataset_results_set:
                                            print(' ~ DMZ Dataset Not Registered: Registering it Now')
                                            
                                            try:    
                                                mysql_dmz_dataset_handler =  mysql_dmz_dataset_connection.cursor()
                                                mysql_dmz_dataset_handler.execute(dmz_dataset_registraton_insert_string)
                                                mysql_dmz_dataset_connection.commit()
                                            
                                                try:
                                                    dbhandler = mysql_dmz_dataset_connection.cursor()
                                                    dbhandler.execute(dmz_dataset_registraton_select_string)
                                                    new_dmz_dataset_results_set = dbhandler.fetchall()
                                                    for new_dmz_dataset_result in new_dmz_dataset_results_set:
                                                        dmz_dataset_key         = str(new_dmz_dataset_result)
                                                        
                                                except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                                                    print('MySQL Error - DMZ Dataset Record Not Found After Registering (via SQL INSERT)')
                                                    print(MySQLexception, MySQLdb.Warning)
                                                    mysql_dmz_dataset_connection.rollback()
                                            
                                            except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                                                    print('MySQL Error - DMZ Dataset Record Registration INSERT Attempted & Failed')
                                                    print(MySQLexception, MySQLdb.Warning)
                                                    mysql_dmz_dataset_connection.rollback()           
                                        else:    
                                            for existing_dmz_dataset_result in existing_dmz_dataset_results_set:
                                                print(' ~ DMZ Dataset Already Registered: ', dmz_dataset_name_tagged)
                                                dmz_dataset_key                 = str(existing_dmz_dataset_result)
                                                
                                    except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                                        print('MySQL Error - DMZ Dataset Lookup Failure - Initial SELECT Statement')
                                        print(MySQLexception, MySQLdb.Warning)
                                    
                                    dmz_dataset_key      = dmz_dataset_key.replace('(', '')
                                    dmz_dataset_key      = dmz_dataset_key.replace(',', '')
                                    dmz_dataset_key      = dmz_dataset_key.replace(')', '')
                                    
                                    print(' ~ ~ DMZ Dataset (' + str(dmz_dataset_key) + '): ' + dmz_dataset_name_tagged + ' \n')
                                    
                                    new_dmz_dataset_array               = numpy.array([dmz_dataset_key,
                                                                                       creator_org_config_master_key,
                                                                                       source_org_config_master_key,
                                                                                       routine_config_master_key,
                                                                                       dmz_dataset_name_raw,
                                                                                       dmz_dataset_name_tagged])
                                    
                                    numpy_new_dmz_dataset_array         = numpy.append(numpy_new_dmz_dataset_array,
                                                                                       [new_dmz_dataset_array], 
                                                                                       axis=0) 
                                    
                                    dmz_dataset_config_notes            = 'Configuration SQL for DMZ Dataset: (' + dmz_dataset_name_tagged + ') Created By: ' + python_protocol + '_' + job_log_suffix
                                    dmz_dataset_config_select_string    = ('SELECT dataset_config_master_key FROM dataset_config_master WHERE dataset_config_master_name = ' + "'" + dmz_dataset_name_tagged + "'")
                                    dmz_dataset_config_insert_string    = ('INSERT INTO dataset_config_master VALUES (NULL , ' + str(277) + ' , '  '0, 0, NULL, CURRENT_TIMESTAMP, NULL' + ', ' + "'" + dmz_dataset_name_tagged + "'" + ', ' + "'" + dmz_dataset_config_notes + "'" + ')')
                                    
                                    new_dmz_dataset_config_array        = numpy.array([dmz_dataset_key, dmz_dataset_config_select_string, dmz_dataset_config_insert_string])
                                    numpy_new_dmz_dataset_config_array  = numpy.append(numpy_new_dmz_dataset_config_array, [new_dmz_dataset_config_array], axis=0)


#
# ~~~ ~~~~ CLASSIFICATION FACETS, VALUES and GRAINS
#

                                    wss_sql_statement_class_facet = class_facet_8000857
                                    wss_sql_statement_cfv_value_alpha   =  hadoop_target_cluster_id
                            
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':
                                        
                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])
                                        
                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)

#
#~~~ ~~~~
#

                                    wss_sql_statement_class_facet = class_facet_8000906
                                    wss_sql_statement_cfv_value_alpha   =  cfv_8000906_FACTA_alpha     
                            
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'N'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)

#
#~~~ #~~~
#

                                    wss_sql_statement_class_facet = class_facet_8000906
                                    wss_sql_statement_cfv_value_alpha   =  cfv_8000906_CPNI_alpha     
                            
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'N'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)


#
#~~~ ~~~~
#


                                    wss_sql_statement_class_facet = class_facet_8000905
                                    wss_sql_statement_cfv_value_alpha   =  metadata_document_description     
                            
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'N'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000894
                                    wss_sql_statement_cfv_value_alpha   = sec_edgar_doc_button_url
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])
        
                                    
                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)

#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000903
                                    wss_sql_statement_cfv_value_alpha   = wss_dataset_collection_name
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000861      
                                    wss_sql_statement_cfv_value_alpha   = hadoop_target_dir_slash_secNum
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])
                                    
                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000862      
                                    wss_sql_statement_cfv_value_alpha   = output_os_directory_cik_prefix
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000880
                                    wss_sql_statement_cfv_value_alpha   = edgar_formName_only
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'N'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000881
                                    wss_sql_statement_cfv_value_alpha   = edgar_secNum_only

                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])
        
                                    
                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    

#
#~~~ ~~~~ NOTICE: Facets 8000881 (above) and 8000876 (below) store the same information. But, User-A says 8000881 is the secNum (SEC Filing Number) and User-B says 800876 is the Accession Number - They are Both Right 
#
                                        
                                    wss_sql_statement_class_facet = class_facet_8000876
                                    wss_sql_statement_cfv_value_alpha   = edgar_secNum_only
                                                                        
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000892      
                                    wss_sql_statement_cfv_value_alpha   = output_os_directory_cik_prefix
                                    
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])
                                    
                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000899      
                                    wss_sql_statement_cfv_value_alpha   = wss_edgar_filing_date
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'N'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])
                                    
                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000900      
                                    wss_sql_statement_cfv_value_alpha   = wss_edgar_filing_accepted
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'N'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])
                                    
                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                                                               

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#
#~~~ ~~~~ NOTICE: With Edgar Filings, 'Documents' is really the number of pages.  Why? Each page is considered a Business Object Document (BOD)
#

                                    wss_sql_statement_class_facet = class_facet_8000901      
                                    wss_sql_statement_cfv_value_alpha   = wss_edgar_filing_documents
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'N'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000902      
                                    wss_sql_statement_cfv_value_alpha   = wss_edgar_filing_period_of_rpt
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'N'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    

#
# ~~~ ~~~~ SHARED CLASSIFICATION FACETS HERE FOR DATASET GRAINS
#
                                    
                                    wss_sql_statement_class_facet = class_facet_8000830
                                    wss_sql_statement_cfv_value_alpha   = cfv_8000830_WindowsServer2012_alpha
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'N'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    
                                    wss_sql_statement_class_facet = class_facet_8000895
                                    wss_sql_statement_cfv_value_alpha   = mysql_01_host
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'N'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    
                                    wss_sql_statement_class_facet = class_facet_8000896
                                    wss_sql_statement_cfv_value_alpha   = str(mysql_01_port)
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'N'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    
                                    wss_sql_statement_class_facet = class_facet_8000897
                                    wss_sql_statement_cfv_value_alpha   = mysql_01_user
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'N'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])
                                    
                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    
                                    wss_sql_statement_class_facet = class_facet_8000898
                                    wss_sql_statement_cfv_value_alpha   = mysql_01_db
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'N'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    
                                    wss_sql_statement_class_facet = class_facet_8000850
                                    wss_sql_statement_cfv_value_alpha   = cfv_8000850_english_alpha
                                    
                                    wss_is_org_config_grain           = 'Y'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    
                                    wss_sql_statement_class_facet = class_facet_8000888
                                    wss_sql_statement_cfv_value_alpha   = python_protocol
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000889      
                                    wss_sql_statement_cfv_value_alpha   = job_log_prefix
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'N'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                        
                                    wss_sql_statement_class_facet = class_facet_8000890      
                                    wss_sql_statement_cfv_value_alpha   = job_log_prefix
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'N'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])
                                    
                                    if facet_value_sql_string not in routine_facet_value_sql_list:
                                        routine_facet_value_sql_list.append(facet_value_sql_string)
                                        dataset_facet_value_sql_list.append(facet_value_sql_string)
                                        numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, [facet_value_sql_array], axis=0)

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                        
                                    wss_sql_statement_class_facet = class_facet_8000870
                                    wss_sql_statement_cfv_value_alpha   = storage_hierarchy_name
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                        
                                    wss_sql_statement_class_facet = class_facet_8000871
                                    wss_sql_statement_cfv_value_alpha   = local_server_os
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'N'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                        
                                    wss_sql_statement_class_facet = class_facet_8000886      
                                    wss_sql_statement_cfv_value_alpha   = local_server_os
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                        
                                    wss_sql_statement_class_facet = class_facet_8000872
                                    wss_sql_statement_cfv_value_alpha   = local_server_ip
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                        
                                    wss_sql_statement_class_facet = class_facet_8000887      
                                    wss_sql_statement_cfv_value_alpha   = local_root_drive
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000873
                                    wss_sql_statement_cfv_value_alpha   = dir_purpose
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000874
                                    wss_sql_statement_cfv_value_alpha   = dir_source
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000875
                                    wss_sql_statement_cfv_value_alpha   = dir_source_subgroup_01
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000877
                                    wss_sql_statement_cfv_value_alpha   = job_log_suffix
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000194
                                    wss_sql_statement_cfv_value_alpha   = cfv_8000194_BeautifulSoup_alpha
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'N'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000194
                                    wss_sql_statement_cfv_value_alpha   = cfv_8000194_datetime_alpha
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'N'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000194
                                    wss_sql_statement_cfv_value_alpha   = cfv_8000194_MySQLdb_alpha
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'N'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000194
                                    wss_sql_statement_cfv_value_alpha   = cfv_8000194_numpy_alpha
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'N'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000194
                                    wss_sql_statement_cfv_value_alpha   = cfv_8000194_os_alpha
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'N'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000194
                                    wss_sql_statement_cfv_value_alpha   = cfv_8000194_pprint_alpha
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'N'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000194
                                    wss_sql_statement_cfv_value_alpha   = cfv_8000194_requests_alpha
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'N'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000194
                                    wss_sql_statement_cfv_value_alpha   = cfv_8000194_urllib_alpha
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'N'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet = class_facet_8000893      
                                    wss_sql_statement_cfv_value_alpha   = sec_edgar_request_url
                                    
                                    wss_is_org_config_grain           = 'N'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#
                                    wss_sql_statement_class_facet         = class_facet_8000885
                                    wss_sql_statement_cfv_value_alpha   = sec_company_cik
                                    
                                    wss_is_org_config_grain           = 'Y'
                                    wss_is_routine_config_grain       = 'Y'
                                    wss_is_dataset_config_grain       = 'Y'

                                    facet_value_select_string = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' + str(wss_sql_statement_class_facet) + ' AND cfv_value_alpha = ' + "'" + wss_sql_statement_cfv_value_alpha + "'")
                                    facet_value_insert_string = ('INSERT INTO chameleon_class_facet_value VALUES (NULL , ' + str(wss_sql_statement_class_facet) + ' , ' + "'" + wss_sql_statement_cfv_value_alpha + "'" + ' , NULL ,NULL ,NULL ,NULL , 0, 0, CURRENT_TIMESTAMP, NULL, NULL)')
                                    facet_value_sql_string    = ('(' + str(wss_sql_statement_class_facet) + ',' + facet_value_select_string + ',' + facet_value_insert_string + ')')
                                    facet_value_sql_array     = numpy.array([str(wss_sql_statement_class_facet), facet_value_select_string , facet_value_insert_string])

                                    if wss_is_routine_config_grain == 'Y':
                                        if facet_value_sql_string not in routine_facet_value_sql_list:
                                            routine_facet_value_sql_list.append(facet_value_sql_string)
                                            
                                            numpy_routine_facet_value_sql_array = numpy.append(numpy_routine_facet_value_sql_array, 
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)                                                         

                                    if wss_is_org_config_grain == 'Y':
                                        if facet_value_sql_string not in org_facet_value_sql_list:
                                            org_facet_value_sql_list.append(facet_value_sql_string)

                                            numpy_org_facet_value_sql_array = numpy.append(numpy_org_facet_value_sql_array,
                                                                                           [facet_value_sql_array],
                                                                                           axis=0)

                                    if wss_is_dataset_config_grain == 'Y':

                                        dataset_facet_value_sql_list.append(facet_value_sql_string)

                                        numpy_dataset_facet_value_sql_array = numpy.append(numpy_dataset_facet_value_sql_array,
                                                                                       [facet_value_sql_array],
                                                                                       axis=0)

                                        new_dmz_dataset_config_grain_array  = numpy.array([dmz_dataset_key,
                                                                                           wss_sql_statement_class_facet,
                                                                                           wss_sql_statement_cfv_value_alpha])

                                        numpy_new_dmz_dataset_config_grain_array = numpy.append(numpy_new_dmz_dataset_config_grain_array,
                                                                                                [new_dmz_dataset_config_grain_array],
                                                                                                axis=0)
                                                                                                    
#
#~~~ ~~~~
#

#
# ~~~ ~~~~ * END * CLASSIFICATION FACETS, VALUES and GRAINS
#



    edgarfetch_log_list.append('_____________________________________________________________________ \n')

    
for numpy_dmz_dataset_registraton_sql_item in numpy_dmz_dataset_registraton_sql_array: 
    edgarfetch_log_list.append('numpy_dmz_dataset_registraton_sql_array')
    edgarfetch_log_list.append('____________________________________ \n')
    edgarfetch_log_list.append(numpy_dmz_dataset_registraton_sql_item)
    
print('_____________________________________________________________________ \n')

mysql_dmz_dataset_connection.close()

#
#~~~ ~~~~ Python Routine Termination
#

print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>          Completed at: ' + str(datetime.datetime.now()))
print('_____________________________________________________________________')
print('_____________________________________________________________________')
print('_____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>          Completed at: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')

#
#~~~ ~~~~ MIT Creative Commons License
#
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------      Chameleon Metadata Python Software     -------------------------------------------
#------------------------------    © 2018 Chameleon Metadata (https://ChameleonMetadata.com)    -------------------------------- 
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------   MIT Creative Commons License   ------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------
# Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated  
# documentation files (the "Software"), to deal in the Software without restriction, including without limitation 
# the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and 
# to permit persons to whom the Software is furnished to do so, subject to the following conditions:
#
# 1) The above copyright notice & this permission notice shall be included in all copies or substantial portions of the Software.
#
# 2) The Software is provided "as is", without warranty of any kind, express or implied, including but not limited to 
#    the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors 
#    or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or 
#    otherwise, arising from, out of or in connection with the software or the use or other dealings in the Software.
#-------------------------------------------------------------------------------------------------------------------------------

#numpy_new_dmz_dataset_config_grain_array.sort(axis=0)
#unique_rows = numpy.unique(numpy_dataset_facet_value_sql_array, axis=0)
#pprint.pprint(numpy_new_dmz_dataset_config_grain_array)

>>>>>
>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S
>>>>>  _______________________________________________
>>>>>
>>>>>         R O U T I N E: EdgarFetch-V5.Receive-Native-Datasets-into-DMZ-Server.18-12-10-12-50
>>>>>            Start Time: 2018-12-10 12:57:08.177857
>>>>>  _____________________________________________________________________ 

_____________________________________________________________________ 

Output Directory Not Found:    x:/dmz-ftp/sec-edgar/0001631574/0001193125-18-157593/18-12-10-12-50
 ~ Output Directory Created:   x:/dmz-ftp/sec-edgar/0001631574/0001193125-18-157593/18-12-10-12-50
wss_edgar_filing_date               =   2018-05-09
wss_edgar_filing_accepted           =   2018-05-09 16:30:03
wss_edgar_filing_documents          =   5
wss_edgar_filing_period_of_rpt      =   2017-12-31
 ~ ~ Writing Edgar Filing Datasets to Target Directory for SEC Accession Number: 0001193125-18-157593
1 :  1
2 :  10-K/A
3 :  d563090d10ka.htm
4 :  10-K/A
5 :  562968
 ~ ~

 ~ ~ DMZ Dataset (15): wve-ex311_8.htm_0001631574_0001564590-18-005308_18-12-10-12-50 

2 :  EX-31.2
3 :  wve-ex312_7.htm
4 :  EX-31.2
5 :  15786
 ~ ~ Onboarding Dataset: x:/dmz-ftp/sec-edgar/0001631574/0001564590-18-005308/18-12-10-12-50/wve-ex312_7.htm
Registering Dataset in DMZ with Name:  wve-ex312_7.htm_0001631574_0001564590-18-005308_18-12-10-12-50
 ~ DMZ Dataset Not Registered: Registering it Now
 ~ ~ DMZ Dataset (16): wve-ex312_7.htm_0001631574_0001564590-18-005308_18-12-10-12-50 

2 :  EX-32
3 :  wve-ex32_6.htm
4 :  EX-32
5 :  8142
 ~ ~ Onboarding Dataset: x:/dmz-ftp/sec-edgar/0001631574/0001564590-18-005308/18-12-10-12-50/wve-ex32_6.htm
Registering Dataset in DMZ with Name:  wve-ex32_6.htm_0001631574_0001564590-18-005308_18-12-10-12-50
 ~ DMZ Dataset Not Registered: Registering it Now
 ~ ~ DMZ Dataset (17): wve-ex32_6.htm_0001631574_0001564590-18-005308_18-12-10-12-50 

2 :  GRAPHIC
3 :  g201803122003056352718.jpg
4 :  GRAPHIC
5 :  22840
 ~ ~ Onboarding Dataset: x:/dmz-ftp/se

 ~ ~ DMZ Dataset (34): g201803122003056702721.jpg_0001631574_0001564590-18-005308_18-12-10-12-50 

2 :  GRAPHIC
3 :  g201803122003054882705.jpg
4 :  GRAPHIC
5 :  7965
 ~ ~ Onboarding Dataset: x:/dmz-ftp/sec-edgar/0001631574/0001564590-18-005308/18-12-10-12-50/g201803122003054882705.jpg
Registering Dataset in DMZ with Name:  g201803122003054882705.jpg_0001631574_0001564590-18-005308_18-12-10-12-50
 ~ DMZ Dataset Not Registered: Registering it Now
 ~ ~ DMZ Dataset (35): g201803122003054882705.jpg_0001631574_0001564590-18-005308_18-12-10-12-50 

2 :  GRAPHIC
3 :  g201803122003049282695.jpg
4 :  GRAPHIC
5 :  13472
 ~ ~ Onboarding Dataset: x:/dmz-ftp/sec-edgar/0001631574/0001564590-18-005308/18-12-10-12-50/g201803122003049282695.jpg
Registering Dataset in DMZ with Name:  g201803122003049282695.jpg_0001631574_0001564590-18-005308_18-12-10-12-50
 ~ DMZ Dataset Not Registered: Registering it Now
 ~ ~ DMZ Dataset (36): g201803122003049282695.jpg_0001631574_0001564590-18-005308_18-12-10-12-50 



 ~ ~ Onboarding Dataset: x:/dmz-ftp/sec-edgar/0001631574/0001564590-18-005308/18-12-10-12-50/wve-20171231_cal.xml
Registering Dataset in DMZ with Name:  wve-20171231_cal.xml_0001631574_0001564590-18-005308_18-12-10-12-50
 ~ DMZ Dataset Not Registered: Registering it Now
 ~ ~ DMZ Dataset (53): wve-20171231_cal.xml_0001631574_0001564590-18-005308_18-12-10-12-50 

2 :  XBRL TAXONOMY EXTENSION DEFINITION LINKBASE
3 :  wve-20171231_def.xml
4 :  EX-101.DEF
5 :  267916
 ~ ~ Onboarding Dataset: x:/dmz-ftp/sec-edgar/0001631574/0001564590-18-005308/18-12-10-12-50/wve-20171231_def.xml
Registering Dataset in DMZ with Name:  wve-20171231_def.xml_0001631574_0001564590-18-005308_18-12-10-12-50
 ~ DMZ Dataset Not Registered: Registering it Now
 ~ ~ DMZ Dataset (54): wve-20171231_def.xml_0001631574_0001564590-18-005308_18-12-10-12-50 

2 :  XBRL TAXONOMY EXTENSION LABEL LINKBASE
3 :  wve-20171231_lab.xml
4 :  EX-101.LAB
5 :  484919
 ~ ~ Onboarding Dataset: x:/dmz-ftp/sec-edgar/0001631574/0001564590-18-

 ~ ~ DMZ Dataset (69): g2017031619163285744647.jpg_0001631574_0001564590-17-004649_18-12-10-12-50 

2 :  GRAPHIC
3 :  g2017031619162448644642.jpg
4 :  GRAPHIC
5 :  18207
 ~ ~ Onboarding Dataset: x:/dmz-ftp/sec-edgar/0001631574/0001564590-17-004649/18-12-10-12-50/g2017031619162448644642.jpg
Registering Dataset in DMZ with Name:  g2017031619162448644642.jpg_0001631574_0001564590-17-004649_18-12-10-12-50
 ~ DMZ Dataset Not Registered: Registering it Now
 ~ ~ DMZ Dataset (70): g2017031619162448644642.jpg_0001631574_0001564590-17-004649_18-12-10-12-50 

2 :  GRAPHIC
3 :  g2017031619162446144641.jpg
4 :  GRAPHIC
5 :  38246
 ~ ~ Onboarding Dataset: x:/dmz-ftp/sec-edgar/0001631574/0001564590-17-004649/18-12-10-12-50/g2017031619162446144641.jpg
Registering Dataset in DMZ with Name:  g2017031619162446144641.jpg_0001631574_0001564590-17-004649_18-12-10-12-50
 ~ DMZ Dataset Not Registered: Registering it Now
 ~ ~ DMZ Dataset (71): g2017031619162446144641.jpg_0001631574_0001564590-17-004649_18-12-1

2 :  XBRL TAXONOMY EXTENSION CALCULATION LINKBASE
3 :  wve-20161231_cal.xml
4 :  EX-101.CAL
5 :  55152
 ~ ~ Onboarding Dataset: x:/dmz-ftp/sec-edgar/0001631574/0001564590-17-004649/18-12-10-12-50/wve-20161231_cal.xml
Registering Dataset in DMZ with Name:  wve-20161231_cal.xml_0001631574_0001564590-17-004649_18-12-10-12-50
 ~ DMZ Dataset Not Registered: Registering it Now
 ~ ~ DMZ Dataset (88): wve-20161231_cal.xml_0001631574_0001564590-17-004649_18-12-10-12-50 

2 :  XBRL TAXONOMY EXTENSION DEFINITION LINKBASE
3 :  wve-20161231_def.xml
4 :  EX-101.DEF
5 :  268794
 ~ ~ Onboarding Dataset: x:/dmz-ftp/sec-edgar/0001631574/0001564590-17-004649/18-12-10-12-50/wve-20161231_def.xml
Registering Dataset in DMZ with Name:  wve-20161231_def.xml_0001631574_0001564590-17-004649_18-12-10-12-50
 ~ DMZ Dataset Not Registered: Registering it Now
 ~ ~ DMZ Dataset (89): wve-20161231_def.xml_0001631574_0001564590-17-004649_18-12-10-12-50 

2 :  XBRL TAXONOMY EXTENSION LABEL LINKBASE
3 :  wve-20161231_lab.

 ~ ~ DMZ Dataset (104): g132744g31u25.jpg_0001631574_0001193125-16-523890_18-12-10-12-50 

2 :  GRAPHIC
3 :  g132744g45d07.jpg
4 :  GRAPHIC
5 :  52270
 ~ ~ Onboarding Dataset: x:/dmz-ftp/sec-edgar/0001631574/0001193125-16-523890/18-12-10-12-50/g132744g45d07.jpg
Registering Dataset in DMZ with Name:  g132744g45d07.jpg_0001631574_0001193125-16-523890_18-12-10-12-50
 ~ DMZ Dataset Not Registered: Registering it Now
 ~ ~ DMZ Dataset (105): g132744g45d07.jpg_0001631574_0001193125-16-523890_18-12-10-12-50 

2 :  GRAPHIC
3 :  g132744g53n07.jpg
4 :  GRAPHIC
5 :  67876
 ~ ~ Onboarding Dataset: x:/dmz-ftp/sec-edgar/0001631574/0001193125-16-523890/18-12-10-12-50/g132744g53n07.jpg
Registering Dataset in DMZ with Name:  g132744g53n07.jpg_0001631574_0001193125-16-523890_18-12-10-12-50
 ~ DMZ Dataset Not Registered: Registering it Now
 ~ ~ DMZ Dataset (106): g132744g53n07.jpg_0001631574_0001193125-16-523890_18-12-10-12-50 

2 :  GRAPHIC
3 :  g132744g59h83.jpg
4 :  GRAPHIC
5 :  35971
 ~ ~ Onboarding D

In [6]:
#~~~    Protocol
#~~~    EdgarFetch
#~~~ ~~~~    Routine
#~~~ ~~~~    Configure-and-Bind-DMZ-Datasets
#

job_log_prefix                 = 'Configure-and-Bind-DMZ-Datasets'
python_routine_long_name       = (python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
                           
print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>            Start Time: ' + str(datetime.datetime.now()))
print('>>>>>  _____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>        Routine Starts: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append(' ')

mysql_dmz_dataset_config_connection = MySQLdb.Connection(host   = mysql_01_host,
                                              port   = mysql_01_port,
                                              user   = mysql_01_user,
                                              passwd = mysql_01_pw,
                                              db     = mysql_01_db)

numpy_dmz_dataset_bind_array = numpy.empty(shape=(0, 2),dtype='int32')

dmz_dataset_bind_list              = []

dmz_dataset_bind_required          = 'N'

for (dmz_dataset_key, sql_select_stmt, sql_insert_stmt) in numpy_new_dmz_dataset_config_array:

    print('_____________________________________________________________________ \n')
    print('dmz_dataset_key: ', dmz_dataset_key)

    dmz_dataset_key      = dmz_dataset_key.replace('(', '')
    dmz_dataset_key      = dmz_dataset_key.replace(',', '')
    dmz_dataset_key      = dmz_dataset_key.replace(')', '')

    try:
        dbhandler = mysql_dmz_dataset_config_connection.cursor()
        dbhandler.execute(sql_select_stmt)
        existing_dmz_dataset_results_set = dbhandler.fetchall()

        if not existing_dmz_dataset_results_set:
            print('Dataset Configuration Master for DMZ Dataset Not Found: Creating It Now')

            try:
                mysql_dataset_config_insert_handler =  mysql_dmz_dataset_config_connection.cursor()
                mysql_dataset_config_insert_handler.execute(sql_insert_stmt)
                mysql_dmz_dataset_config_connection.commit()
                dmz_dataset_bind_required          = 'Y'

                try:
                    mysql_new_dataset_config_handler = mysql_dmz_dataset_config_connection.cursor()
                    mysql_new_dataset_config_handler.execute(sql_select_stmt)
                    new_dataset_config_results_set = mysql_new_dataset_config_handler.fetchall()
                    for new_dataset_config_result in new_dataset_config_results_set:
                        dataset_config_master_key       = str(new_dataset_config_result)

                except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                    print('MySQL Error - Dataset Configuration Master Record Not Found After INSERT')
                    print(MySQLexception, MySQLdb.Warning)
                    mysql_dmz_dataset_config_connection.rollback()

            except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                    print('MySQL Error - Dataset Configuration Master Record INSERT Attempted & Failed')
                    print(MySQLexception, MySQLdb.Warning)
                    mysql_dmz_dataset_config_connection.rollback()
        else:
            for existing_dmz_dataset_result in existing_dmz_dataset_results_set:
                print('Dataset Configuration Master for DMZ Dataset Already Exists: ')
                print('dataset_config_master_key Set To: ', str(existing_dmz_dataset_result))
                dataset_config_master_key       = str(existing_dmz_dataset_result)

    except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
        print('MySQL Error - Dataset Configuration Master Lookup Failure - Initial SELECT Statement')
        print(MySQLexception, MySQLdb.Warning)

    dataset_config_master_key      = dataset_config_master_key.replace('(', '')
    dataset_config_master_key      = dataset_config_master_key.replace(',', '')
    dataset_config_master_key      = dataset_config_master_key.replace(')', '')

    dmz_dataset_config_bind_array           = numpy.array([int(dmz_dataset_key), int(dataset_config_master_key)])

    if dmz_dataset_config_bind_array not in numpy_dmz_dataset_bind_array:
        numpy_dmz_dataset_bind_array     = numpy.append(numpy_dmz_dataset_bind_array, [dmz_dataset_config_bind_array], axis=0)

    dmz_dataset_auto_bind_notes         = '(DMZ Dataset: ' + str(dmz_dataset_key) + ') Auto-Bind to (Dataset Config: ' + str(dataset_config_master_key) + ') Done By: ' + python_protocol + '_' + job_log_suffix
    dmz_dataset_auto_bind_insert_string = ('INSERT INTO dataset_config_bind VALUES (NULL , ' + str(dataset_config_master_key) + ' , '  + str(dmz_dataset_key) + ' , '  +  'NULL, 0, 0, CURRENT_TIMESTAMP, NULL, NULL, ' + "'" + dmz_dataset_auto_bind_notes + "'" + ')')

    dmz_dataset_bind_list.append(dmz_dataset_auto_bind_insert_string)
    edgarfetch_log_list.append('(DMZ Dataset: ' + str(dmz_dataset_key) + ') Auto-Bind to (Dataset Config: ' + str(dataset_config_master_key) + ') Done By: ' + python_protocol + '_' + job_log_suffix)
    edgarfetch_log_list.append(' \n')


    if dmz_dataset_bind_required         == 'Y':
        try:
            mysql_dataset_bind_handler =  mysql_dmz_dataset_config_connection.cursor()
            mysql_dataset_bind_handler.execute(dmz_dataset_auto_bind_insert_string)
            mysql_dmz_dataset_config_connection.commit()
            print('Dataset Bind (DMZ Dataset:', str(dmz_dataset_key), ') << Now Bound To >> (Dataset Config: ', str(dataset_config_master_key) ,')')
            dmz_dataset_bind_required          = 'N'

        except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
            print('MySQL Error - Dataset Configuration Bind Attempted for DMZ Dataset: (' + str(dmz_dataset_key) + '<< BIND TO >> Dataset Config: ' + str(dataset_config_master_key) + ') Attempted & Failed')
            print(MySQLexception, MySQLdb.Warning)
            mysql_dmz_dataset_config_connection.rollback()

mysql_dmz_dataset_config_connection.close()

edgarfetch_log_list.append(' \n')
edgarfetch_log_list.append('numpy_dmz_dataset_bind_array')
edgarfetch_log_list.append('([[ dmz_dataset_key,  dataset_config_master_key]]) \n')
edgarfetch_log_list.append(numpy_dmz_dataset_bind_array)

edgarfetch_log_list.append('_____________________________________________________________________ \n')


print('_____________________________________________________________________ \n')

#
#~~~ ~~~~ Python Routine Termination
#

print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>          Completed at: ' + str(datetime.datetime.now()))
print('_____________________________________________________________________')
print('_____________________________________________________________________')
print('_____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>          Completed at: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')

#
#~~~ ~~~~ MIT Creative Commons License
#
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------      Chameleon Metadata Python Software     -------------------------------------------
#------------------------------    © 2018 Chameleon Metadata (https://ChameleonMetadata.com)    -------------------------------- 
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------   MIT Creative Commons License   ------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------
# Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated  
# documentation files (the "Software"), to deal in the Software without restriction, including without limitation 
# the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and 
# to permit persons to whom the Software is furnished to do so, subject to the following conditions:
#
# 1) The above copyright notice & this permission notice shall be included in all copies or substantial portions of the Software.
#
# 2) The Software is provided "as is", without warranty of any kind, express or implied, including but not limited to 
#    the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors 
#    or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or 
#    otherwise, arising from, out of or in connection with the software or the use or other dealings in the Software.
#-------------------------------------------------------------------------------------------------------------------------------


>>>>>
>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S
>>>>>  _______________________________________________
>>>>>
>>>>>         R O U T I N E: EdgarFetch-V5.Configure-and-Bind-DMZ-Datasets.18-12-10-12-50
>>>>>            Start Time: 2018-12-10 12:59:11.126267
>>>>>  _____________________________________________________________________ 

_____________________________________________________________________ 

dmz_dataset_key:  1
Dataset Configuration Master for DMZ Dataset Not Found: Creating It Now
Dataset Bind (DMZ Dataset: 1 ) << Now Bound To >> (Dataset Config:  1 )
_____________________________________________________________________ 

dmz_dataset_key:  2
Dataset Configuration Master for DMZ Dataset Not Found: Creating It Now
Dataset Bind (DMZ Dataset: 2 ) << Now Bound To >> (Dataset Config:  2 )
_____________________________________________________________________ 

dmz_dataset_key:  3
Dataset Configuration Master for DMZ Dataset Not Found: Creating It Now
Dataset Bind (DMZ

Dataset Bind (DMZ Dataset: 34 ) << Now Bound To >> (Dataset Config:  34 )
_____________________________________________________________________ 

dmz_dataset_key:  35
Dataset Configuration Master for DMZ Dataset Not Found: Creating It Now
Dataset Bind (DMZ Dataset: 35 ) << Now Bound To >> (Dataset Config:  35 )
_____________________________________________________________________ 

dmz_dataset_key:  36
Dataset Configuration Master for DMZ Dataset Not Found: Creating It Now
Dataset Bind (DMZ Dataset: 36 ) << Now Bound To >> (Dataset Config:  36 )
_____________________________________________________________________ 

dmz_dataset_key:  37
Dataset Configuration Master for DMZ Dataset Not Found: Creating It Now
Dataset Bind (DMZ Dataset: 37 ) << Now Bound To >> (Dataset Config:  37 )
_____________________________________________________________________ 

dmz_dataset_key:  38
Dataset Configuration Master for DMZ Dataset Not Found: Creating It Now
Dataset Bind (DMZ Dataset: 38 ) << Now Bound

Dataset Bind (DMZ Dataset: 69 ) << Now Bound To >> (Dataset Config:  69 )
_____________________________________________________________________ 

dmz_dataset_key:  70
Dataset Configuration Master for DMZ Dataset Not Found: Creating It Now
Dataset Bind (DMZ Dataset: 70 ) << Now Bound To >> (Dataset Config:  70 )
_____________________________________________________________________ 

dmz_dataset_key:  71
Dataset Configuration Master for DMZ Dataset Not Found: Creating It Now
Dataset Bind (DMZ Dataset: 71 ) << Now Bound To >> (Dataset Config:  71 )
_____________________________________________________________________ 

dmz_dataset_key:  72
Dataset Configuration Master for DMZ Dataset Not Found: Creating It Now
Dataset Bind (DMZ Dataset: 72 ) << Now Bound To >> (Dataset Config:  72 )
_____________________________________________________________________ 

dmz_dataset_key:  73
Dataset Configuration Master for DMZ Dataset Not Found: Creating It Now
Dataset Bind (DMZ Dataset: 73 ) << Now Bound

Dataset Bind (DMZ Dataset: 104 ) << Now Bound To >> (Dataset Config:  104 )
_____________________________________________________________________ 

dmz_dataset_key:  105
Dataset Configuration Master for DMZ Dataset Not Found: Creating It Now
Dataset Bind (DMZ Dataset: 105 ) << Now Bound To >> (Dataset Config:  105 )
_____________________________________________________________________ 

dmz_dataset_key:  106
Dataset Configuration Master for DMZ Dataset Not Found: Creating It Now
Dataset Bind (DMZ Dataset: 106 ) << Now Bound To >> (Dataset Config:  106 )
_____________________________________________________________________ 

dmz_dataset_key:  107
Dataset Configuration Master for DMZ Dataset Not Found: Creating It Now
Dataset Bind (DMZ Dataset: 107 ) << Now Bound To >> (Dataset Config:  107 )
_____________________________________________________________________ 

dmz_dataset_key:  108
Dataset Configuration Master for DMZ Dataset Not Found: Creating It Now
Dataset Bind (DMZ Dataset: 108 )

In [7]:
#~~~    Protocol
#~~~    EdgarFetch
#~~~ ~~~~    Routine
#~~~ ~~~~    Confirm-or-Create-Org-Class-Facet-Values
#

job_log_prefix                 = 'Confirm-or-Create-Org-Class-Facet-Values'
python_routine_long_name       = (python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
                           
print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>            Start Time: ' + str(datetime.datetime.now()))
print('>>>>>  _____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>        Routine Starts: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append(' ')


mysql_org_facets_connection = MySQLdb.Connection(host      = mysql_01_host, 
                                                    port   = mysql_01_port,
                                                    user   = mysql_01_user, 
                                                    passwd = mysql_01_pw, 
                                                    db     = mysql_01_db)

numpy_org_facet_value_array = numpy.empty(shape=(0, 2),dtype='int32')


for (classification_facet_key, sql_select_stmt, sql_insert_stmt) in numpy_org_facet_value_sql_array:
    
    try:    
        mysql_org_facets_dbhandler = mysql_org_facets_connection.cursor()
        mysql_org_facets_dbhandler.execute(sql_select_stmt)
        existing_org_cf_results_set = mysql_org_facets_dbhandler.fetchall()

        if not existing_org_cf_results_set:
            print('Classification Facet Value Not Found: Creating It Now')
            
            try:    
                mysql_new_org_cf_handler =  mysql_org_facets_connection.cursor()
                mysql_new_org_cf_handler.execute(sql_insert_stmt)
                mysql_org_facets_connection.commit()
            
                try:
                    mysql_new_org_cf_handler = mysql_org_facets_connection.cursor()
                    mysql_new_org_cf_handler.execute(sql_select_stmt)
                    new_org_cf_results_set = mysql_new_org_cf_handler.fetchall()
                    for new_org_cf_result in new_org_cf_results_set:
                        print('Classification Facet Value Set To: ', str(new_org_cf_result), ' \n')
                        string_results_item = str(new_org_cf_result)
                                                
                except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                    print('MySQL Error - Classification Facet Value Record Not Found After INSERT')
                    print(MySQLexception, MySQLdb.Warning)
                    mysql_org_facets_connection.rollback()
            
            except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                    print('MySQL Error - Classification Facet Value Record INSERT Attempted & Failed')
                    print(MySQLexception, MySQLdb.Warning)
                    mysql_org_facets_connection.rollback()           
        else:    
            for existing_org_cf_result in existing_org_cf_results_set:                              
                print('Classification Facet Value Already Exists: ')
                print('Classification Facet Value Set To: ', str(existing_org_cf_result), ' \n')
                
                string_results_item = str(existing_org_cf_result)
                
    except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
        print('MySQL Error - Classification Facet Value Lookup Failure - Initial SELECT Statement')
        print(MySQLexception, MySQLdb.Warning)

    class_facet_value_key     = str(string_results_item)
    class_facet_value_key     = class_facet_value_key.replace(')', '')
    class_facet_value_key     = class_facet_value_key.replace('(', '')
    class_facet_value_key     = class_facet_value_key.replace(',', '')
    
    
    org_facet_value_array           = numpy.array([int(classification_facet_key), int(class_facet_value_key)])
    
    if org_facet_value_array not in numpy_org_facet_value_array:
        numpy_org_facet_value_array     = numpy.append(numpy_org_facet_value_array, [org_facet_value_array], axis=0)

mysql_org_facets_connection.close()

#
#~~~ ~~~~ Python Routine Termination
#

print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>          Completed at: ' + str(datetime.datetime.now()))
print('_____________________________________________________________________')
print('_____________________________________________________________________')
print('_____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>          Completed at: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')

#
#~~~ ~~~~ MIT Creative Commons License
#
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------      Chameleon Metadata Python Software     -------------------------------------------
#------------------------------    © 2018 Chameleon Metadata (https://ChameleonMetadata.com)    -------------------------------- 
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------   MIT Creative Commons License   ------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------
# Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated  
# documentation files (the "Software"), to deal in the Software without restriction, including without limitation 
# the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and 
# to permit persons to whom the Software is furnished to do so, subject to the following conditions:
#
# 1) The above copyright notice & this permission notice shall be included in all copies or substantial portions of the Software.
#
# 2) The Software is provided "as is", without warranty of any kind, express or implied, including but not limited to 
#    the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors 
#    or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or 
#    otherwise, arising from, out of or in connection with the software or the use or other dealings in the Software.
#-------------------------------------------------------------------------------------------------------------------------------


>>>>>
>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S
>>>>>  _______________________________________________
>>>>>
>>>>>         R O U T I N E: EdgarFetch-V5.Confirm-or-Create-Org-Class-Facet-Values.18-12-10-12-50
>>>>>            Start Time: 2018-12-10 13:00:25.636629
>>>>>  _____________________________________________________________________ 

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7016098,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019072,)  

>>>>>
>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D
>>>>>  _______________________________________________
>>>>>
>>>>>         R O U T I N E: EdgarFetch-V5.Confirm-or-Create-Org-Class-Facet-Values.18-12-10-12-50
>>>>>          Completed at: 2018-12-10 13:00:26.160842
_____________________________________________________________________
_____________________________________________________________________
_________________________________

In [8]:
#~~~    Protocol
#~~~    EdgarFetch
#~~~ ~~~~    Routine
#~~~ ~~~~    Confirm-or-Create-Routine-Class-Facet-Values
#

job_log_prefix                 = 'Confirm-or-Create-Routine-Class-Facet-Values'
python_routine_long_name       = (python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
                           
print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>            Start Time: ' + str(datetime.datetime.now()))
print('>>>>>  _____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>        Routine Starts: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append(' ')

mysql_routine_facets_connection = MySQLdb.Connection(host   = mysql_01_host, 
                                                    port   = mysql_01_port,
                                                    user   = mysql_01_user, 
                                                    passwd = mysql_01_pw, 
                                                    db     = mysql_01_db)

numpy_routine_facet_value_array = numpy.empty(shape=(0, 2),dtype='int32')


for (classification_facet_key, sql_select_stmt, sql_insert_stmt) in numpy_routine_facet_value_sql_array:
    
    try:    
        mysql_routine_facets_dbhandler = mysql_routine_facets_connection.cursor()
        mysql_routine_facets_dbhandler.execute(sql_select_stmt)
        existing_routine_cf_results_set = mysql_routine_facets_dbhandler.fetchall()

        if not existing_routine_cf_results_set:
            print('Classification Facet Value Not Found: Creating It Now')
            
            try:    
                mysql_new_routine_cf_handler =  mysql_routine_facets_connection.cursor()
                mysql_new_routine_cf_handler.execute(sql_insert_stmt)
                mysql_routine_facets_connection.commit()
            
                try:
                    mysql_new_routine_cf_handler = mysql_routine_facets_connection.cursor()
                    mysql_new_routine_cf_handler.execute(sql_select_stmt)
                    new_routine_cf_results_set = mysql_new_routine_cf_handler.fetchall()
                    for new_routine_cf_result in new_routine_cf_results_set:
                        print('Classification Facet Value Set To: ', str(new_routine_cf_result), ' \n')
                        string_results_item = str(new_routine_cf_result)
                                                
                except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                    print('MySQL Error - Classification Facet Value Record Not Found After INSERT')
                    print(MySQLexception, MySQLdb.Warning)
                    mysql_routine_facets_connection.rollback()
            
            except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                    print('MySQL Error - Classification Facet Value Record INSERT Attempted & Failed')
                    print(MySQLexception, MySQLdb.Warning)
                    mysql_routine_facets_connection.rollback()           
        else:    
            for existing_routine_cf_result in existing_routine_cf_results_set:                              
                print('Classification Facet Value Already Exists: ')
                print('Classification Facet Value Set To: ', str(existing_routine_cf_result), ' \n')
                
                string_results_item = str(existing_routine_cf_result)
                
    except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
        print('MySQL Error - Classification Facet Value Lookup Failure - Initial SELECT Statement')
        print(MySQLexception, MySQLdb.Warning)

    class_facet_value_key     = str(string_results_item)
    class_facet_value_key     = class_facet_value_key.replace(')', '')
    class_facet_value_key     = class_facet_value_key.replace('(', '')
    class_facet_value_key     = class_facet_value_key.replace(',', '')
    
    
    routine_facet_value_array           = numpy.array([int(classification_facet_key), int(class_facet_value_key)])
    numpy_routine_facet_value_array     = numpy.append(numpy_routine_facet_value_array, [routine_facet_value_array], axis=0)

mysql_routine_facets_connection.close()

#
#~~~ ~~~~ Python Routine Termination
#

print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>          Completed at: ' + str(datetime.datetime.now()))
print('_____________________________________________________________________')
print('_____________________________________________________________________')
print('_____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>          Completed at: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')

#
#~~~ ~~~~ MIT Creative Commons License
#
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------      Chameleon Metadata Python Software     -------------------------------------------
#------------------------------    © 2018 Chameleon Metadata (https://ChameleonMetadata.com)    -------------------------------- 
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------   MIT Creative Commons License   ------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------
# Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated  
# documentation files (the "Software"), to deal in the Software without restriction, including without limitation 
# the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and 
# to permit persons to whom the Software is furnished to do so, subject to the following conditions:
#
# 1) The above copyright notice & this permission notice shall be included in all copies or substantial portions of the Software.
#
# 2) The Software is provided "as is", without warranty of any kind, express or implied, including but not limited to 
#    the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors 
#    or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or 
#    otherwise, arising from, out of or in connection with the software or the use or other dealings in the Software.
#-------------------------------------------------------------------------------------------------------------------------------


>>>>>
>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S
>>>>>  _______________________________________________
>>>>>
>>>>>         R O U T I N E: EdgarFetch-V5.Confirm-or-Create-Routine-Class-Facet-Values.18-12-10-12-50
>>>>>            Start Time: 2018-12-10 13:00:34.816681
>>>>>  _____________________________________________________________________ 

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019542,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019104,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019838,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019106,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019107,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019073,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set T

In [9]:
#~~~    Protocol
#~~~    EdgarFetch
#~~~ ~~~~    Routine
#~~~ ~~~~    Confirm-or-Create-Dataset-Class-Facet-Values
#

job_log_prefix                 = 'Confirm-or-Create-Dataset-Class-Facet-Values'
python_routine_long_name       = (python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
                           
print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>            Start Time: ' + str(datetime.datetime.now()))
print('>>>>>  _____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>        Routine Starts: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append(' ')


mysql_dataset_facets_connection = MySQLdb.Connection(host   = mysql_01_host, 
                                                    port    = mysql_01_port,
                                                    user    = mysql_01_user, 
                                                    passwd  = mysql_01_pw, 
                                                    db      = mysql_01_db)

numpy_dataset_facet_value_array = numpy.empty(shape=(0, 2),dtype='int32')


for (classification_facet_key, sql_select_stmt, sql_insert_stmt) in numpy_dataset_facet_value_sql_array:
    
    try:    
        mysql_dataset_facets_dbhandler = mysql_dataset_facets_connection.cursor()
        mysql_dataset_facets_dbhandler.execute(sql_select_stmt)
        existing_dataset_cf_results_set = mysql_dataset_facets_dbhandler.fetchall()
        
        if not existing_dataset_cf_results_set:
            print('Classification Facet Value Not Found: Creating It Now')
            print(' ~ Facet (' + str(classification_facet_key)  + '): ' + sql_insert_stmt)
            
            try:    
                mysql_new_dataset_cf_handler =  mysql_dataset_facets_connection.cursor()
                mysql_new_dataset_cf_handler.execute(sql_insert_stmt)
                mysql_dataset_facets_connection.commit()
            
                try:
                    mysql_new_dataset_cf_handler = mysql_dataset_facets_connection.cursor()
                    mysql_new_dataset_cf_handler.execute(sql_select_stmt)
                    new_dataset_cf_results_set = mysql_new_dataset_cf_handler.fetchall()
                    for new_dataset_cf_result in new_dataset_cf_results_set:
                        print('Classification Facet Value Set To: ', str(new_dataset_cf_result), ' \n')
                        string_results_item = str(new_dataset_cf_result)
                                                
                except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                    print('MySQL Error - Classification Facet Value Record Not Found After INSERT')
                    print(' ~ Facet (' + str(classification_facet_key)  + '): ' + sql_insert_stmt)
                    
                    print(MySQLexception, MySQLdb.Warning)
                    mysql_dataset_facets_connection.rollback()
            
            except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                    print('MySQL Error - Classification Facet Value Record INSERT Attempted & Failed')
                    print(' ~ Facet (' + str(classification_facet_key)  + '): ' + sql_insert_stmt)
                    
                    print(MySQLexception, MySQLdb.Warning)
                    mysql_dataset_facets_connection.rollback()           
        else:    
            for existing_dataset_cf_result in existing_dataset_cf_results_set:                              
                print('Classification Facet Value Already Exists: ')
                print('Classification Facet Value Set To: ', str(existing_dataset_cf_result), ' \n')
                
                string_results_item = str(existing_dataset_cf_result)
                
    except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
        print('MySQL Error - Classification Facet Value Lookup Failure - Initial SELECT Statement')
        print(' ~ Facet (' + str(classification_facet_key)  + '): ' + sql_select_stmt)
        print(MySQLexception, MySQLdb.Warning)

    class_facet_value_key     = str(string_results_item)
    class_facet_value_key     = class_facet_value_key.replace(')', '')
    class_facet_value_key     = class_facet_value_key.replace('(', '')
    class_facet_value_key     = class_facet_value_key.replace(',', '')
    
    
    dataset_facet_value_array           = numpy.array([int(classification_facet_key), int(class_facet_value_key)])
    
    if dataset_facet_value_array not in numpy_dataset_facet_value_array:
        numpy_dataset_facet_value_array     = numpy.append(numpy_dataset_facet_value_array, [dataset_facet_value_array], axis=0)

mysql_dataset_facets_connection.close()

#
#~~~ ~~~~ Python Routine Termination
#

print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>          Completed at: ' + str(datetime.datetime.now()))
print('_____________________________________________________________________')
print('_____________________________________________________________________')
print('_____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>          Completed at: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')

#
#~~~ ~~~~ MIT Creative Commons License
#
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------      Chameleon Metadata Python Software     -------------------------------------------
#------------------------------    © 2018 Chameleon Metadata (https://ChameleonMetadata.com)    -------------------------------- 
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------   MIT Creative Commons License   ------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------
# Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated  
# documentation files (the "Software"), to deal in the Software without restriction, including without limitation 
# the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and 
# to permit persons to whom the Software is furnished to do so, subject to the following conditions:
#
# 1) The above copyright notice & this permission notice shall be included in all copies or substantial portions of the Software.
#
# 2) The Software is provided "as is", without warranty of any kind, express or implied, including but not limited to 
#    the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors 
#    or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or 
#    otherwise, arising from, out of or in connection with the software or the use or other dealings in the Software.
#-------------------------------------------------------------------------------------------------------------------------------


>>>>>
>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S
>>>>>  _______________________________________________
>>>>>
>>>>>         R O U T I N E: EdgarFetch-V5.Confirm-or-Create-Dataset-Class-Facet-Values.18-12-10-12-50
>>>>>            Start Time: 2018-12-10 13:00:56.136071
>>>>>  _____________________________________________________________________ 

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019542,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019857,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019854,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019732,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019104,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019838,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set T

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019542,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019857,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019854,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019712,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019104,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019838,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019106,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019107,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019128,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019073,)  

Classification Facet Value Already Exists: 
Classification F

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019111,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019107,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019128,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019075,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019076,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019108,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019129,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019130,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019131,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019132,)  

Classification Facet Value Already Exists: 
Classification F

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019129,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019130,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019131,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019132,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7016098,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019747,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019094,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019096,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019097,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019098,)  

Classification Facet Value Already Exists: 
Classification F

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019094,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019096,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019097,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019098,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019099,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019100,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019101,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019858,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019749,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019072,)  

Classification Facet Value Already Exists: 
Classification F

...

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7016098,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019747,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019094,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019096,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019097,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019098,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019099,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019100,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019101,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019858,)  

Classification Facet Value Already Exists: 
Classification F

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019099,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019100,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019101,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019858,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019749,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019072,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019542,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019857,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019854,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019706,)  

Classification Facet Value Already Exists: 
Classification F

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019542,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019857,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019854,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019706,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019124,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019845,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019126,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019107,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019142,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019085,)  

Classification Facet Value Already Exists: 
Classification F

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019126,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019107,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019142,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019085,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019086,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019108,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019129,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019130,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019131,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019132,)  

Classification Facet Value Already Exists: 
Classification F

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019129,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019130,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019131,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019132,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7016098,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019747,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019094,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019096,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019097,)  

Classification Facet Value Already Exists: 
Classification Facet Value Set To:  (7019098,)  

Classification Facet Value Already Exists: 
Classification F

In [10]:
#~~~    Protocol
#~~~    EdgarFetch
#~~~ ~~~~    Routine
#~~~ ~~~~    Confirm-or-Create-Organization-Config-Grains
#

job_log_prefix                 = 'Confirm-or-Create-Organization-Config-Grains'
python_routine_long_name       = (python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
                           
print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>            Start Time: ' + str(datetime.datetime.now()))
print('>>>>>  _____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>        Routine Starts: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append(' ')

mysql_org_grains_connection = MySQLdb.Connection(host   = mysql_01_host, 
                                                    port    = mysql_01_port,
                                                    user    = mysql_01_user, 
                                                    passwd  = mysql_01_pw, 
                                                    db      = mysql_01_db)

numpy_org_config_grain_array     = numpy.empty(shape=(0, 4),dtype='int32')

org_grain_insert_list            = []

sql_select_string                = ' '
sql_insert_string                = ' '
string_org_grains_item           = ' '

for (org_class_facet_key, org_facet_value_key) in numpy_org_facet_value_array:
    for (org_master_key,  org_config_master_key) in numpy_creator_org_config_bind_array:
        
        sql_select_string   = 'SELECT org_config_grain_key FROM organization_config_grain WHERE  org_config_master_key = ' +  str(org_config_master_key) + ' AND  class_facet_value_key = ' + str(org_facet_value_key)
        sql_insert_stmt     = ('INSERT INTO organization_config_grain VALUES (NULL , ' +  str(org_config_master_key) + ', ' + str(org_class_facet_key) + ', ' + str(org_facet_value_key) +  ' , 0, CURRENT_TIMESTAMP, NULL, NULL)')

        try:    
            mysql_org_grain_handler =  mysql_org_grains_connection.cursor()
            mysql_org_grain_handler.execute(sql_select_string)
            existing_org_grain_results_set = mysql_org_grain_handler.fetchall()
            
            if not existing_org_grain_results_set:
                print('Organization Configuration Grain Not Found: Creating It Now')
                
                try:    
                    mysql_new_org_grain_handler =  mysql_org_grains_connection.cursor()
                    mysql_new_org_grain_handler.execute(sql_insert_stmt)
                    mysql_org_grains_connection.commit()
                    
                    try:
                        mysql_org_grain_handler =  mysql_org_grains_connection.cursor()
                        mysql_org_grain_handler.execute(sql_select_string)
                        existing_org_grain_results_set = mysql_org_grain_handler.fetchall()
                        for existing_org_grain_result in existing_org_grain_results_set:
                            print('Organization Grain Key Set To: ', str(existing_org_grain_result), ' \n')
                            string_org_grains_item = str(existing_org_grain_result)

                    except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                        print('MySQL Error - New Organization Grain Key Not Found After INSERT')
                        print(MySQLexception, MySQLdb.Warning)
                        mysql_org_grains_connection.rollback()
                
                except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                    print('MySQL Error - New Organization Grain Record INSERT Attempted & Failed')
                    print(MySQLexception, MySQLdb.Warning)
                    mysql_org_grains_connection.rollback()
            else:    
                for existing_org_grain_result in existing_org_grain_results_set:                              
                    print('Organization Grain Record Already Exists with Key: ', str(existing_org_grain_result))
                    print('org_grain_key: ', str(existing_org_grain_result), ' . ', 'cm_key: ',str(org_config_master_key), ' . ', 'cf_key: ',str(org_class_facet_key), ' . ', 'cfv_key: ',str(org_facet_value_key), ' . ', ' \n')
                    string_org_grains_item = str(existing_org_grain_result)
        
        except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
            print('MySQL Error - Classification Facet Value Lookup Failure - Initial SELECT Statement')
            print(MySQLexception, MySQLdb.Warning)
        
        
        org_config_grain_key     = str(string_org_grains_item)
        org_config_grain_key     = org_config_grain_key.replace(')', '')
        org_config_grain_key     = org_config_grain_key.replace('(', '')
        org_config_grain_key     = org_config_grain_key.replace(',', '')
        
        
        print('****', org_config_grain_key, org_config_master_key, org_class_facet_key, org_facet_value_key)
        
        org_grain_array              = numpy.array([int(org_config_grain_key), int(org_config_master_key), int(org_class_facet_key), int(org_facet_value_key)])
        numpy_org_config_grain_array = numpy.append(numpy_org_config_grain_array, [org_grain_array], axis=0)

        
mysql_org_grains_connection.close()

#
#~~~ ~~~~ Python Routine Termination
#

print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>          Completed at: ' + str(datetime.datetime.now()))
print('_____________________________________________________________________')
print('_____________________________________________________________________')
print('_____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>          Completed at: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')

#
#~~~ ~~~~ MIT Creative Commons License
#
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------      Chameleon Metadata Python Software     -------------------------------------------
#------------------------------    © 2018 Chameleon Metadata (https://ChameleonMetadata.com)    -------------------------------- 
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------   MIT Creative Commons License   ------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------
# Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated  
# documentation files (the "Software"), to deal in the Software without restriction, including without limitation 
# the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and 
# to permit persons to whom the Software is furnished to do so, subject to the following conditions:
#
# 1) The above copyright notice & this permission notice shall be included in all copies or substantial portions of the Software.
#
# 2) The Software is provided "as is", without warranty of any kind, express or implied, including but not limited to 
#    the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors 
#    or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or 
#    otherwise, arising from, out of or in connection with the software or the use or other dealings in the Software.
#-------------------------------------------------------------------------------------------------------------------------------


>>>>>
>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S
>>>>>  _______________________________________________
>>>>>
>>>>>         R O U T I N E: EdgarFetch-V5.Confirm-or-Create-Organization-Config-Grains.18-12-10-12-50
>>>>>            Start Time: 2018-12-10 13:06:03.328294
>>>>>  _____________________________________________________________________ 

Organization Grain Record Already Exists with Key:  (7,)
org_grain_key:  (7,)  .  cm_key:  3300087  .  cf_key:  8000850  .  cfv_key:  7016098  .   

**** 7 3300087 8000850 7016098
Organization Grain Record Already Exists with Key:  (8,)
org_grain_key:  (8,)  .  cm_key:  3300087  .  cf_key:  8000885  .  cfv_key:  7019072  .   

**** 8 3300087 8000885 7019072
>>>>>
>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D
>>>>>  _______________________________________________
>>>>>
>>>>>         R O U T I N E: EdgarFetch-V5.Confirm-or-Create-Organization-Config-Grains.18-12-10-12-50
>>>>>          Completed at: 2018-12-10 13:06:03.

In [11]:
#~~~    Protocol
#~~~    EdgarFetch
#~~~ ~~~~    Routine
#~~~ ~~~~    Confirm-or-Create-Routine-Config-Grains
#

job_log_prefix                 = 'Confirm-or-Create-Routine-Config-Grains'
python_routine_long_name       = (python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
                           
print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>            Start Time: ' + str(datetime.datetime.now()))
print('>>>>>  _____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>        Routine Starts: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append(' ')

mysql_routine_grains_connection = MySQLdb.Connection(host   = mysql_01_host, 
                                                    port    = mysql_01_port,
                                                    user    = mysql_01_user, 
                                                    passwd  = mysql_01_pw, 
                                                    db      = mysql_01_db)

numpy_routine_config_grain_array     = numpy.empty(shape=(0, 4),dtype='int32')

routine_grain_insert_list            = []

sql_select_string                    = ' '
sql_insert_string                    = ' '
string_routine_grains_item           = ' '

for (routine_class_facet_key, routine_facet_value_key) in numpy_routine_facet_value_array:
    for (routine_master_key,  routine_config_master_key) in numpy_routine_config_bind_array:
        
        sql_select_string   = 'SELECT routine_config_grain_key FROM routine_config_grain WHERE  routine_config_master_key = ' +  str(routine_config_master_key) + ' AND  class_facet_value_key = ' + str(routine_facet_value_key)
        sql_insert_stmt     = ('INSERT INTO routine_config_grain VALUES (NULL , ' +  str(routine_config_master_key) + ', ' + str(routine_class_facet_key) + ', ' + str(routine_facet_value_key) +  ' , 0, CURRENT_TIMESTAMP, NULL, NULL)')

        try:    
            mysql_routine_grain_handler =  mysql_routine_grains_connection.cursor()
            mysql_routine_grain_handler.execute(sql_select_string)
            existing_routine_grain_results_set = mysql_routine_grain_handler.fetchall()
            
            if not existing_routine_grain_results_set:
                print('Routine Configuration Grain Not Found: Creating It Now')
                
                try:    
                    mysql_new_routine_grain_handler =  mysql_routine_grains_connection.cursor()
                    mysql_new_routine_grain_handler.execute(sql_insert_stmt)
                    mysql_routine_grains_connection.commit()
                    
                    try:
                        mysql_routine_grain_handler =  mysql_routine_grains_connection.cursor()
                        mysql_routine_grain_handler.execute(sql_select_string)
                        existing_routine_grain_results_set = mysql_routine_grain_handler.fetchall()
                        for existing_routine_grain_result in existing_routine_grain_results_set:
                            print('Routine Grain Key Set To: ', str(existing_routine_grain_result), ' \n')
                            string_routine_grains_item = str(existing_routine_grain_result)

                    except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                        print('MySQL Error - New Routine Grain Key Not Found After INSERT')
                        print(MySQLexception, MySQLdb.Warning)
                        mysql_routine_grains_connection.rollback()
                
                except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                    print('MySQL Error - New Routine Grain Record INSERT Attempted & Failed')
                    print(MySQLexception, MySQLdb.Warning)
                    mysql_routine_grains_connection.rollback()
            else:    
                for existing_routine_grain_result in existing_routine_grain_results_set:                              
                    print('Routine Grain Record Already Exists with Key: ', str(existing_routine_grain_result))
                    print('routine_grain_array Record:', routine_grain_array, ' \n')
                    string_routine_grains_item = str(existing_routine_grain_result)
        
        except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
            print('MySQL Error - Classification Facet Value Lookup Failure - Initial SELECT Statement')
            print(MySQLexception, MySQLdb.Warning)
        
        
        routine_config_grain_key     = str(string_routine_grains_item)
        routine_config_grain_key     = routine_config_grain_key.replace(')', '')
        routine_config_grain_key     = routine_config_grain_key.replace('(', '')
        routine_config_grain_key     = routine_config_grain_key.replace(',', '')
        
        
                               
        routine_grain_array       = numpy.array([int(routine_config_grain_key), int(routine_config_master_key), int(routine_class_facet_key), int(routine_facet_value_key)])
        numpy_routine_config_grain_array = numpy.append(numpy_routine_config_grain_array, [routine_grain_array], axis=0)

        
mysql_routine_grains_connection.close()

#
#~~~ ~~~~ Python Routine Termination
#

print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>          Completed at: ' + str(datetime.datetime.now()))
print('_____________________________________________________________________')
print('_____________________________________________________________________')
print('_____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>          Completed at: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')

#
#~~~ ~~~~ MIT Creative Commons License
#
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------      Chameleon Metadata Python Software     -------------------------------------------
#------------------------------    © 2018 Chameleon Metadata (https://ChameleonMetadata.com)    -------------------------------- 
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------   MIT Creative Commons License   ------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------
# Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated  
# documentation files (the "Software"), to deal in the Software without restriction, including without limitation 
# the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and 
# to permit persons to whom the Software is furnished to do so, subject to the following conditions:
#
# 1) The above copyright notice & this permission notice shall be included in all copies or substantial portions of the Software.
#
# 2) The Software is provided "as is", without warranty of any kind, express or implied, including but not limited to 
#    the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors 
#    or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or 
#    otherwise, arising from, out of or in connection with the software or the use or other dealings in the Software.
#-------------------------------------------------------------------------------------------------------------------------------


>>>>>
>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S
>>>>>  _______________________________________________
>>>>>
>>>>>         R O U T I N E: EdgarFetch-V5.Confirm-or-Create-Routine-Config-Grains.18-12-10-12-50
>>>>>            Start Time: 2018-12-10 13:06:14.667808
>>>>>  _____________________________________________________________________ 

Routine Configuration Grain Not Found: Creating It Now
Routine Grain Key Set To:  (1,)  

Routine Configuration Grain Not Found: Creating It Now
Routine Grain Key Set To:  (2,)  

Routine Configuration Grain Not Found: Creating It Now
Routine Grain Key Set To:  (3,)  

Routine Configuration Grain Not Found: Creating It Now
Routine Grain Key Set To:  (4,)  

Routine Configuration Grain Not Found: Creating It Now
Routine Grain Key Set To:  (5,)  

Routine Configuration Grain Not Found: Creating It Now
Routine Grain Key Set To:  (6,)  

Routine Configuration Grain Not Found: Creating It Now
Routine Grain Key Set To:  (7,)  

Routine Configu

In [12]:
#~~~    Protocol
#~~~    EdgarFetch
#~~~ ~~~~    Routine
#~~~ ~~~~    Confirm-or-Create-Dataset-Config-Grains
#

job_log_prefix                 = 'Confirm-or-Create-Dataset-Config-Grains'
python_routine_long_name       = (python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
                           
print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>            Start Time: ' + str(datetime.datetime.now()))
print('>>>>>  _____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>        Routine Starts: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append(' ')

mysql_dataset_grains_connection = MySQLdb.Connection(host   = mysql_01_host, 
                                                    port    = mysql_01_port,
                                                    user    = mysql_01_user, 
                                                    passwd  = mysql_01_pw, 
                                                    db      = mysql_01_db)

numpy_dataset_config_grain_array      = numpy.empty(shape=(0, 4),dtype='int32')

dataset_grain_insert_list             = []

dataset_cfv_key_lookup_sql_list       = []


dataset_class_facet_value_lookup_sql  = ' '
dataset_grain_select_string_sql       = ' '
dataset_grain_insert_string_sql       = ' '

wss_steward_person_party_key          = 29

wss_class_facet_value_key             = ' '
wss_class_facet_value_alpha           = ' '

wss_dataset_config_master_key         = ' '
wss_dataset_config_grain_key          = ' '

for (grain_dmz_dataset_key, wss_sql_statement_class_facet, wss_class_facet_value_alpha) in numpy_new_dmz_dataset_config_grain_array:
    for (bind_dmz_dataset_key, bind_dataset_config_master_key) in numpy_dmz_dataset_bind_array:

#~~~ ~~~~ BEGIN: if int(grain_dmz_dataset_key) == int(bind_dmz_dataset_key)

        if int(grain_dmz_dataset_key) == int(bind_dmz_dataset_key):

            dataset_class_facet_value_lookup_sql   = ('SELECT class_facet_value_key FROM chameleon_class_facet_value WHERE classification_facet_key = ' 
                                                      + str(wss_sql_statement_class_facet) 
                                                      + ' AND  cfv_value_alpha = ' 
                                                      + "'" + wss_class_facet_value_alpha + "'")

            try:
                mysql_dataset_cfv_lookup_handler =  mysql_dataset_grains_connection.cursor()                                        
                mysql_dataset_cfv_lookup_handler.execute(dataset_class_facet_value_lookup_sql)                                      
                existing_dataset_cfv_lookup_results_set = mysql_dataset_cfv_lookup_handler.fetchone()                               
                                                                                                                                    
                if not existing_dataset_cfv_lookup_results_set:                                                                     
                                                                                                                                    
                    print('****************************************************************************')                           
                    print('***     A  L  E  R  T    ***    A  L  E  R  T    ***    A  L  E  R  T    ***')                           
                    print('****************************************************************************')                           
                    print(' ')                                                                                                      
                    print('class_facet_value_key Not Found During Lookup: This is a serious error')                                 
                    print(' ~ ', dataset_class_facet_value_lookup_sql)                                                              
                    print(' ')                                                                                                      
                    print(' ~ ~ Many times this error is the result of a bad point-in-time recovery')                               
                    print(' ~ ~ Class Facets, Facet Values and Config Grains must be properly aligneded')                           
                    print(' ')                                                                                                      
                    print('****************************************************************************')                           
                    print(' ')                                                                                                      
                                                                                                                                    
                    edgarfetch_log_list.append('****************************************************************************')      
                    edgarfetch_log_list.append('***     A  L  E  R  T    ***    A  L  E  R  T    ***    A  L  E  R  T    ***')      
                    edgarfetch_log_list.append('****************************************************************************')      
                    edgarfetch_log_list.append(' ')                                                                                 
                    edgarfetch_log_list.append('class_facet_value_key Not Found During Lookup: This is a serious error')            
                    edgarfetch_log_list.append(' ~ ' + dataset_class_facet_value_lookup_sql)                                        
                    edgarfetch_log_list.append(' ')                                                                                 
                    edgarfetch_log_list.append(' ~ ~ Many times this error is the result of a bad point-in-time recovery')          
                    edgarfetch_log_list.append(' ~ ~ Class Facets, Facet Values and Config Grains must be properly aligneded')      
                    edgarfetch_log_list.append(' ')                                                                                 
                    edgarfetch_log_list.append('****************************************************************************')      
                    edgarfetch_log_list.append(' ')                                                                                 
                                                                                                                                    
                else:                                                                                                               
                    for existing_dataset_cfv_lookup_result in existing_dataset_cfv_lookup_results_set:                              
                        print('Class Facet Value Key Lookup Successful: ', str(existing_dataset_cfv_lookup_result))                 
                                                                                                                                    
                        wss_class_facet_value_key = existing_dataset_cfv_lookup_result                                              

            except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                print('MySQL Error - Class Facet Value Key Lookup Attempted & Failed')
                print(' ~ (' + dataset_class_facet_value_lookup_sql + ')')
                print(MySQLexception, MySQLdb.Warning)
                mysql_dataset_grains_connection.rollback()           


            dataset_grain_select_string_sql = ('SELECT dataset_config_grain_key FROM dataset_config_grain WHERE dataset_config_master_key = ' 
                                               +  str(bind_dataset_config_master_key) 
                                               + ' AND  class_facet_value_key = ' 
                                               + str(wss_class_facet_value_key))
            
            dataset_grain_insert_string_sql = ('INSERT INTO dataset_config_grain VALUES(NULL' + ', ' 
                                               + str(bind_dataset_config_master_key) + ', '
                                               + str(wss_sql_statement_class_facet) + ', '
                                               + str(wss_class_facet_value_key) + ', '
                                               + str(wss_steward_person_party_key) + ', ' 
                                               +  'CURRENT_TIMESTAMP, NULL, NULL)')

            try:    
                mysql_dataset_grains_dbhandler = mysql_dataset_grains_connection.cursor()
                mysql_dataset_grains_dbhandler.execute(dataset_grain_select_string_sql)
                existing_dataset_grain_results_set = mysql_dataset_grains_dbhandler.fetchall()
                
                
                if not existing_dataset_grain_results_set:
                    print('Dataset Config Grain Not Found: Creating It Now')
                    print(' ~ ', dataset_grain_insert_string_sql, ' \n')
                    
                    try:    
                        mysql_new_dataset_grain_handler =  mysql_dataset_grains_connection.cursor()
                        mysql_new_dataset_grain_handler.execute(dataset_grain_insert_string_sql)
                        mysql_dataset_grains_connection.commit()
                    
                        try:
                            mysql_new_dataset_grain_handler = mysql_dataset_grains_connection.cursor()
                            mysql_new_dataset_grain_handler.execute(dataset_grain_select_string_sql)
                            new_dataset_grain_results_set = mysql_new_dataset_grain_handler.fetchall()
                            for new_dataset_grain_result in new_dataset_grain_results_set:
                                print('Dataset Config Grain Key Set To: ', str(new_dataset_grain_result), ' \n')
                                string_results_item = str(new_dataset_grain_result)
                                                        
                        except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                            print('MySQL Error - Dataset Config Grain Record Not Found After INSERT')
                            print(' ~ Facet (' + str(classification_facet_key)  + '): ' + sql_insert_stmt)
                            
                            print(MySQLexception, MySQLdb.Warning)
                            mysql_dataset_grains_connection.rollback()
                    
                    except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                            print('MySQL Error - Dataset Config Grain Record INSERT Attempted & Failed')
                            print(' ~ Facet (' + str(classification_facet_key)  + '): ' + sql_insert_stmt)
                            
                            print(MySQLexception, MySQLdb.Warning)
                            mysql_dataset_grains_connection.rollback()           
                else:    
                    for existing_dataset_grain_result in existing_dataset_grain_results_set:                              
                        print('Dataset Config Grain Already Exists: ')
                        print(' ~  Dataset Config Grain Key Set To: ', str(existing_dataset_grain_result), ' \n')
                        
            except (MySQLdb.Error, MySQLdb.Warning) as MySQLexception:
                print('MySQL Error - Dataset Config Grain Lookup Failure - Initial SELECT Statement')
                print(' ~ Facet (' + str(classification_facet_key)  + '): ' + sql_select_stmt)
                print(MySQLexception, MySQLdb.Warning)

#~~~ ~~~~ ENDIF (no 'else' clause): int(grain_dmz_dataset_key) == int(bind_dmz_dataset_key)

mysql_dataset_grains_connection.close()

#
#~~~ ~~~~ Python Routine Termination
#

print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>          Completed at: ' + str(datetime.datetime.now()))
print('_____________________________________________________________________')
print('_____________________________________________________________________')
print('_____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>          Completed at: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')

#
#~~~ ~~~~ MIT Creative Commons License
#
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------      Chameleon Metadata Python Software     -------------------------------------------
#------------------------------    © 2018 Chameleon Metadata (https://ChameleonMetadata.com)    -------------------------------- 
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------   MIT Creative Commons License   ------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------
# Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated  
# documentation files (the "Software"), to deal in the Software without restriction, including without limitation 
# the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and 
# to permit persons to whom the Software is furnished to do so, subject to the following conditions:
#
# 1) The above copyright notice & this permission notice shall be included in all copies or substantial portions of the Software.
#
# 2) The Software is provided "as is", without warranty of any kind, express or implied, including but not limited to 
#    the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors 
#    or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or 
#    otherwise, arising from, out of or in connection with the software or the use or other dealings in the Software.
#-------------------------------------------------------------------------------------------------------------------------------


>>>>>
>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S
>>>>>  _______________________________________________
>>>>>
>>>>>         R O U T I N E: EdgarFetch-V5.Confirm-or-Create-Dataset-Config-Grains.18-12-10-12-50
>>>>>            Start Time: 2018-12-10 13:06:55.718360
>>>>>  _____________________________________________________________________ 

Class Facet Value Key Lookup Successful:  7019542
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 1, 8000857, 7019542, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (1,)  

Class Facet Value Key Lookup Successful:  7019857
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 1, 8000906, 7019857, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (2,)  

Class Facet Value Key Lookup Successful:  7019854
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL,

Dataset Config Grain Key Set To:  (32,)  

Class Facet Value Key Lookup Successful:  7019104
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 2, 8000894, 7019104, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (33,)  

Class Facet Value Key Lookup Successful:  7019838
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 2, 8000903, 7019838, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (34,)  

Class Facet Value Key Lookup Successful:  7019106
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 2, 8000861, 7019106, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (35,)  

Class Facet Value Key Lookup Successful:  7019107
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 2, 8000862, 7019107, 29, CURRENT_TIMESTAMP, NULL, NULL)  



Dataset Config Grain Key Set To:  (65,)  

Class Facet Value Key Lookup Successful:  7019073
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 3, 8000881, 7019073, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (66,)  

Class Facet Value Key Lookup Successful:  7019074
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 3, 8000876, 7019074, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (67,)  

Class Facet Value Key Lookup Successful:  7019108
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 3, 8000892, 7019108, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (68,)  

Class Facet Value Key Lookup Successful:  7019129
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 3, 8000899, 7019129, 29, CURRENT_TIMESTAMP, NULL, NULL)  



Dataset Config Grain Key Set To:  (98,)  

Class Facet Value Key Lookup Successful:  7019131
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 4, 8000901, 7019131, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (99,)  

Class Facet Value Key Lookup Successful:  7019132
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 4, 8000902, 7019132, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (100,)  

Class Facet Value Key Lookup Successful:  7016098
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 4, 8000850, 7016098, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (101,)  

Class Facet Value Key Lookup Successful:  7019747
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 4, 8000888, 7019747, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (131,)  

Class Facet Value Key Lookup Successful:  7019096
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 5, 8000886, 7019096, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (132,)  

Class Facet Value Key Lookup Successful:  7019097
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 5, 8000872, 7019097, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (133,)  

Class Facet Value Key Lookup Successful:  7019098
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 5, 8000887, 7019098, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (134,)  

Class Facet Value Key Lookup Successful:  7019099
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 5, 8000873, 7019099, 29, CURRENT_TIMESTAMP, NULL, NULL)

...

Dataset Config Grain Key Set To:  (3200,)  

Class Facet Value Key Lookup Successful:  7019142
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 115, 8000880, 7019142, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (3201,)  

Class Facet Value Key Lookup Successful:  7019085
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 115, 8000881, 7019085, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (3202,)  

Class Facet Value Key Lookup Successful:  7019086
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 115, 8000876, 7019086, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (3203,)  

Class Facet Value Key Lookup Successful:  7019108
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 115, 8000892, 7019108, 29, CURRENT_TIMESTAMP,

Dataset Config Grain Key Set To:  (3233,)  

Class Facet Value Key Lookup Successful:  7019130
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 116, 8000900, 7019130, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (3234,)  

Class Facet Value Key Lookup Successful:  7019131
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 116, 8000901, 7019131, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (3235,)  

Class Facet Value Key Lookup Successful:  7019132
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 116, 8000902, 7019132, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (3236,)  

Class Facet Value Key Lookup Successful:  7016098
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 116, 8000850, 7016098, 29, CURRENT_TIMESTAMP,

Dataset Config Grain Key Set To:  (3266,)  

Class Facet Value Key Lookup Successful:  7019094
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 117, 8000870, 7019094, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (3267,)  

Class Facet Value Key Lookup Successful:  7019096
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 117, 8000886, 7019096, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (3268,)  

Class Facet Value Key Lookup Successful:  7019097
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 117, 8000872, 7019097, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (3269,)  

Class Facet Value Key Lookup Successful:  7019098
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 117, 8000887, 7019098, 29, CURRENT_TIMESTAMP,

Dataset Config Grain Key Set To:  (3299,)  

Class Facet Value Key Lookup Successful:  7019100
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 118, 8000874, 7019100, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (3300,)  

Class Facet Value Key Lookup Successful:  7019101
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 118, 8000875, 7019101, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (3301,)  

Class Facet Value Key Lookup Successful:  7019858
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 118, 8000877, 7019858, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (3302,)  

Class Facet Value Key Lookup Successful:  7019749
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 118, 8000893, 7019749, 29, CURRENT_TIMESTAMP,

Dataset Config Grain Key Set To:  (3332,)  

Class Facet Value Key Lookup Successful:  7019542
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 120, 8000857, 7019542, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (3333,)  

Class Facet Value Key Lookup Successful:  7019857
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 120, 8000906, 7019857, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (3334,)  

Class Facet Value Key Lookup Successful:  7019854
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 120, 8000906, 7019854, 29, CURRENT_TIMESTAMP, NULL, NULL)  

Dataset Config Grain Key Set To:  (3335,)  

Class Facet Value Key Lookup Successful:  7019719
Dataset Config Grain Not Found: Creating It Now
 ~  INSERT INTO dataset_config_grain VALUES(NULL, 120, 8000905, 7019719, 29, CURRENT_TIMESTAMP,

In [23]:
#~~~    Protocol
#~~~    EdgarFetch
#~~~ ~~~~    Routine
#~~~ ~~~~    EdgarFetch-Write-Flat-Files
#

job_log_prefix                 = 'EdgarFetch-Write-Flat-Files'
python_routine_long_name       = (python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
                           
print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>            Start Time: ' + str(datetime.datetime.now()))
print('>>>>>  _____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   B E G I N S')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>        Routine Starts: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append(' ')

#
#~~~ ~~~~ 2.3.3 Write Flat File: EdgarFetch-Filings_YY-MM-DD-HH-MM_Edgar.reply
#
with open(sec_edgar_reply_url_file, 'w') as file_request:
    for sec_edgar_doc_buttons_url in sec_edgar_doc_buttons_url_list:
        file_request.write(sec_edgar_doc_buttons_url)
        file_request.write(' \n')
        
file_request.close()

#
#~~~ ~~~~ 2.3.4 Write Flat File: output_os_replyDocButtons_file
#
with open(output_os_replyDocButtons_file, 'w') as file_request:
    for reply_metadata_item in sec_edgar_reply_metadata_list:
        file_request.write(reply_metadata_item)
        file_request.write(' \n')
        
file_request.close()

#
#~~~ ~~~~ 2.3.5 Write Flat File: output_os_replyDocsMetadata_file
#
with open(output_os_replyDocsMetadata_file, 'w') as file_request:
    for document_metadata_item in sec_edgar_document_metadata_list:
        file_request.write(document_metadata_item)
        file_request.write(' \n')
    
file_request.close()

#
#~~~ ~~~~ 2.3.6 Write Flat File: output_os_replyDocsURL_file
#
with open(output_os_replyDocsURL_file, 'w') as file_request:
    for sec_edgar_document_item in sec_edgar_document_url_list:
        file_request.write(sec_edgar_document_item)
        file_request.write(' \n')
    
file_request.close()

#
#~~~ ~~~~ 2.3.7 Write Flat File: output_os_hadoop_mkdir_cmds_file
#
with open(output_os_hadoop_mkdir_cmds_file, 'w') as file_request:
    for hadoop_mkdir_command_item in hadoop_mkdir_command_list:
        file_request.write(hadoop_mkdir_command_item)
        file_request.write(' \n')
    
file_request.close()

#
#~~~ ~~~~ 2.3.8 Write Flat File: output_os_hadoop_copy_cmds_file
#
with open(output_os_hadoop_copy_cmds_file, 'w') as file_request:
    for hadoop_copy_command_item in hadoop_copy_command_list:
        file_request.write(hadoop_copy_command_item)
        file_request.write(' \n')
    
file_request.close()

#
#~~~ ~~~~ 2.3.9 Write Flat Files: Routine, Dataset and Org facet_value_sql_file
#

with open(routine_facet_value_sql_file, 'w') as file_request:
    for (class_facet_key, sql_select_stmt, sql_insert_stmt) in numpy_routine_facet_value_sql_array:
        file_request.write(class_facet_key + ', ' + sql_select_stmt + ', ' + sql_insert_stmt)
        file_request.write(' \n')
        
file_request.close()

org_facet_value_sql_list.sort()
with open(creator_org_facet_value_sql_file, 'w') as file_request:
    for sql_facet_value_sql_statement in org_facet_value_sql_list:
        file_request.write(sql_facet_value_sql_statement)
        file_request.write(' \n')
        
file_request.close()

dataset_facet_value_sql_list.sort()
with open(dataset_facet_value_sql_file, 'w') as file_request:
    for sql_facet_value_sql_statement in dataset_facet_value_sql_list:
        file_request.write(sql_facet_value_sql_statement)
        file_request.write(' \n')
        
file_request.close()

#~~~ #~~~ #~~~ #~~~

routine_config_sql_list.sort()
with open(routine_config_sql_file, 'w') as file_request:
    for sql_routine_config_sql_statement in routine_config_sql_list:
        file_request.write(sql_routine_config_sql_statement)
        file_request.write(' \n')
        
file_request.close()

creator_org_config_sql_list.sort()
with open(org_config_sql_file, 'w') as file_request:
    for sql_org_config_sql_statement in creator_org_config_sql_list:
        file_request.write(sql_org_config_sql_statement)
        file_request.write(' \n')
        
file_request.close()

#~~~ #~~~ #~~~ #~~~

with open(edgar_fetch_log_file, 'w') as file_request:
    for edgar_fetch_log_item in edgarfetch_log_list:
        file_request.write(str(edgar_fetch_log_item))
        file_request.write(' \n')
        
file_request.close()
#
#~~~ ~~~~ Python Routine Termination
#

print('>>>>>')
print('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
print('>>>>>  _______________________________________________')
print('>>>>>')
print('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
print('>>>>>          Completed at: ' + str(datetime.datetime.now()))
print('_____________________________________________________________________')
print('_____________________________________________________________________')
print('_____________________________________________________________________ \n')

edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>  R O U T I N E   E X E C U T I O N   C O M P L E T E D')
edgarfetch_log_list.append('>>>>>  _______________________________________________')
edgarfetch_log_list.append('>>>>>')
edgarfetch_log_list.append('>>>>>         R O U T I N E: ' + python_protocol + '.' + job_log_prefix + '.' + job_log_suffix)
edgarfetch_log_list.append('>>>>>          Completed at: ' + str(datetime.datetime.now()))
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')
edgarfetch_log_list.append('_____________________________________________________________________')

#
#~~~ ~~~~ MIT Creative Commons License
#
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------      Chameleon Metadata Python Software     -------------------------------------------
#------------------------------    © 2018 Chameleon Metadata (https://ChameleonMetadata.com)    -------------------------------- 
#-------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------   MIT Creative Commons License   ------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------
# Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated  
# documentation files (the "Software"), to deal in the Software without restriction, including without limitation 
# the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and 
# to permit persons to whom the Software is furnished to do so, subject to the following conditions:
#
# 1) The above copyright notice & this permission notice shall be included in all copies or substantial portions of the Software.
#
# 2) The Software is provided "as is", without warranty of any kind, express or implied, including but not limited to 
#    the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors 
#    or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or 
#    otherwise, arising from, out of or in connection with the software or the use or other dealings in the Software.
#-------------------------------------------------------------------------------------------------------------------------------


<-<-<-<-<-<-<-<-<-------  E X E C U T I O N   B E G I N S  ------->->->->->->->->->->
_____________________________________________________________________ 

>>>>>         R O U T I N E: EdgarFetch-V5.EdgarFetch-Write-Flat-Files
>>>>>             Start Time: 2018-12-10 14:17:41.267058
_____________________________________________________________________ 

_____________________________________________________________________ 

>>>>>         R O U T I N E: EdgarFetch-V5.EdgarFetch-Write-Flat-Files
>>>>>              Finsihed: 2018-12-10 14:17:41.384090                             <<<<<
_____________________________________________________________________ 

